# Dependencias

In [1]:
from utils import *
from layer_generator import *
from dict_utils import *
from list_utils import *
from txt_utils import *
from PORT_MAP_utils import *
from vhd_txt_utils import *
from tree_utils import *
from layer_utils import *
from name import *
from standard_dicts import *
from MAC import *
import pandas as pd
import numpy as np


### <font color=green>  **Parâmetros Gerais**  </font>

In [2]:
gera_automatico = 0                # 1 = gera | 0 = não gera

bits_list_gen = [4,6,8,10]          #lista de bits para versões
inputs_list_gen = [4,8,16,32,64]    #lista de inputs ...


DEBUG = 1              # 1 = mostra outputs no jupyter| 0 = não mostra


if (gera_automatico == 1):

  
  #-------------------- Loop listas ------------------------------#
  if (inputs_list_index >= len(inputs_list_gen) -1): #Quando chega no final da lista de 'inputs'
    inputs_list_index = 0
    bits_list_index += 1      # Quando ainda não é o final da lista de 'bits'
    
  else: # Quando ainda não é o final da lista de 'inputs'
    inputs_list_index += 1
    
  if ( bits_list_index >= len(bits_list_gen)): #Quando chega no final da lista de 'bits'
    bits_list_index = 0
    
  bits = bits_list_gen[bits_list_index]                #define o número de bits para as entradas e pesos
  num_inputs = inputs_list_gen[inputs_list_index]      #define número de inputs      

  #------------------------- Sem Loop ----------------------------#
else:  
  bits = 8                #define o número de bits para as entradas e pesos
  num_inputs = 3          #número de entradas e pesos do perceptron
  
#-------------------------------------------------------------------------------------------------#
  
saidas_debug = 0        #1= para colocar saidas nos .vhd || 0= para não colocar saídas de debug
IO_type = 1             #1= signed || 0= unsigned
memory_mux= 0           #1= ter mux no somatório para não buscar na memória sempre || 0= não ter -> busca sempre na memória

leaky_attenuation = 2   #número de vezes que divide por 2 a ReLU (shift right)
# activ_fx_middle = 1     # 1= ReLU | 2 = Leaky_Relu


MAC_type = 0            #0 = combinational(tree:árvore) | 1 = Sequential
Include_MAC_type = 1    #0 = don't incluce | 1 = include 'seq' or 'tree' on vhd_names
Barriers = 1            #0 = no Barriers | 1 = with register Barriers (pipeline)

bits_mem = 8
input_mem_bits = bits_mem
output_mem_bits = bits_mem

n_bin = bits # 'n' binary digits are the fractional part of `x`; = MANTISSA

download_vhd =  0       #1= para baixar || 0= não baixar
download_txt =  0       #1= para baixar || 0= não baixar

download_files =  (download_vhd or download_txt)     #1= para baixar || 0= não baixar


In [3]:
if (IO_type == 1):      #versão com sinal
  IO_type = 'signed'
  std_logic_conversion = IO_type
else:                     #versão sem sinal
  IO_type = 'STD_LOGIC_VECTOR'
  std_logic_conversion = 'unsigned'

In [4]:
# gera_automatico = 0

if (gera_automatico == 1):
      print(" bits: "+  str(bits_list_gen[bits_list_index]),
            "\n inputs: "+ str(inputs_list_gen[inputs_list_index ]),
            "\n bits_index: "+ str(bits_list_index),
            "\n inputs_index: "+ str(inputs_list_index + 1))

In [5]:
# https://colab.research.google.com/drive/12xCU8oNStmXCekDzL-P9hx7ro-obPTUu#scrollTo=GDrWC4BP0gIu
m_bin = bits - n_bin # `m` binary digits are the integer part
# m_bin = 5 # `m` binary digits are the integer part
# n_bin = n_bits - m_bin # 'n' binary digits are the fractional part of `x`;
n_bin_sigmoid = n_bin *2 

sup_threshold= 2**m_bin
inf_threshold= 0

### <font color=green>    **Criando Pasta**   </font>

In [6]:
Neurons_hidden = ['ReLU','Leaky_ReLU']
Neurons_softmax = ['Sigmoid']

path, path_soft = create_folder_neuron(bits, num_inputs, Neurons_hidden,Neurons_softmax,MAC_type,Barriers,1,1)
# create_folder_neuron(Neurons_hidden,Neurons_softmax,MAC_type,Barriers,1,download_files)
# (hidden,softmax,MAC_type,Barriers,path_level,create)
  # MAC_type:     0 = combinacional         | 1 = Sequential
  # has_pipe:     0 = no Barriers           | 1 = with register Barriers (pipeline)
  # path_level:   0 = Same level path            | 1 = Above path
  # create:       0 = Don't Create Folders  | 1 = Create Folders 

path_soft = path_soft[0]
path_ReLU = path[0]
path_LeakyReLU = path[1]

if (DEBUG == 1 ):
  print(f"Relu: {path[0]}")
  print('----------------------')
  print(f"Leaky Relu: {path[1]}")
  print('----------------------')
  print(f"Softmax: {path_soft}")

Created Folders!!
Relu: c:\Users\Jairo Luiz\OneDrive\Documentos\Luis Spader\Programming\NN_generator/Neuron/HiddenLayer/Tree/BARRIERS/ReLU/8bit/3n
----------------------
Leaky Relu: c:\Users\Jairo Luiz\OneDrive\Documentos\Luis Spader\Programming\NN_generator/Neuron/HiddenLayer/Tree/BARRIERS/Leaky_ReLU/8bit/3n
----------------------
Softmax: c:\Users\Jairo Luiz\OneDrive\Documentos\Luis Spader\Programming\NN_generator/Neuron/SoftmaxLayer/Tree/BARRIERS/Sigmoid/8bit/3n


In [7]:
# import os
# for i in range(0,len(Neurons_softmax)):    
#   os.makedirs(f"{path_soft[i]}", exist_ok=True) # softmax layer
#   print(path_soft[i])
# # for i in range(0,len(hidden_layer_neurons)):
# #   os.makedirs(f"{path[i]}", exist_ok=True)   # hidden layer

In [8]:
# Pasta testbench
project_folder_path = (r'''"C:/Users/luisa/Documents/Materias/UFSC/20201/Lab_EBD_ECL/Perpeptron/NN_Neuron_unsigned_no_components_generic_inputs_python_ROM_2"''')

# Dicionários

## <font color=green>    **Neuron**    </font>

In [9]:
prefix_neuron = 'neuron_'
fx_activation_dict = {
  'ReLU' : f'{prefix_neuron}ReLU',
  'Leaky ReLU': f'{prefix_neuron}LeakyReLU',
  'Sigmoid' : f'{prefix_neuron}Sigmoid'
}

dict_list = dict_to_list(target_dict = fx_activation_dict, key_or_value = True) # fazendo uma lista do dicionário
# ['ReLU', 'Leaky ReLU', 'Sigmoid']

Neuron_arch_dict = {  
  'Inputs_number': 3, #número de entradas e pesos do perceptron
  'Bit_WIDTH': 8,     #define o número de bits para as entradas e pesos
  'IO_type': True,    # True= signed || 0= unsigned
  
  # --------------------------
  'IO':{ # INPUT & OUTPUT
    'IN': { # ENTRADAS
      'STD_LOGIC': ['clk', 'rst'],
      'STD_LOGIC_VECTOR': None,
      'SIGNED': ['bias'],
      # depende do número de entradas = número de neurônios da camada anterior
      'STD_LOGIC_num_inputs': None,
      'STD_LOGIC_VECTOR_num_inputs': None,
      'SIGNED_num_inputs': ['x']
      },
    'OUT': { #SAÍDAS
      'STD_LOGIC': None,
      'STD_LOGIC_VECTOR': None,
      'SIGNED': ['y'],
      # depende do número de entradas
      'STD_LOGIC_VECTOR_num_inputs': None,
      'STD_LOGIC_num_inputs': None,
      'SIGNED_num_inputs': None,
      # depende do número de neurônios
      'STD_LOGIC_VECTOR_num_neurons': None,
      'STD_LOGIC_num_neurons': None,
      'SIGNED_num_neurons': None
      }
  },
  
  # -------------
  'MAC_type':{
    'Combinational': True,
    'Sequential': False
  },
  
  # -------------
  'Barriers': False,  # True = com barreiras de registradores
  
  # ------------- Versões de multiplicador
  'Multiplier':{
    'Operator': True, # *
    'Baugh_Wooley': {
      'Alexandre': False,
      'Luis': False
      },
    'Booth': False,
    'Shift': False,
    'Quantized_shift': False
    },
  
  # -------------
  'Adder':{
    'Operator': True # +
    },
  
  # -------------
  'Activation_function':{
    'ReLU': True,
    'Leaky_ReLU': {
      'Using': False,        # True = usar versão Leaky_ReLU
      'Leaky_attenuation': 2 #número de vezes que divide por 2 a ReLU (shift right)
    },
    'Sigmoid': {
      'Using': False,        # True = usar versão Sigmoid (Look Up Table)
      'Memory': {
        'bits_mem': 8,
        'input_mem_bits': lambda:Neuron_arch_dict['Sigmoid']['Memory']['bits_mem'],  # 'n' binary digits are the fractional part of `x`; = MANTISSA
      }
    }
  },
  
  # ------------- dict_list 0: ReLU, 1: Leaky ReLU, 2: Sigmoid
  'neuron_name': vhd_name(dict_list[0], bits, IO_type, num_inputs, Barriers, MAC_type, Include_MAC_type,
                          mult_number = 0,
                          mult_version = 0,
                          adder_number = 0,
                          adder_version = 0)
}


In [10]:
Neuron_IO_dict = {
  'IN': { # ENTRADAS
    'STD_LOGIC': ['clk', 'rst'],
    'STD_LOGIC_VECTOR': None,
    'STD_LOGIC_num_inputs': None,
    'STD_LOGIC_VECTOR_num_inputs': None,
    'SIGNED': ['bias'],
    'SIGNED_num_inputs': ['x']
    },
 'OUT': { #SAÍDAS
   'STD_LOGIC': None,
    'STD_LOGIC_VECTOR': None,
    'STD_LOGIC_num_inputs': None,
    'STD_LOGIC_VECTOR_num_inputs': None,
    'SIGNED': ['y'],
    'SIGNED_num_inputs': None
    }
 }
# Neuron_IO_dict

In [11]:
dict_test = Neuron_IO_dict
# dict_test['IN']['STD_LOGIC'] = ['ABC']  OK
dict_test['IN']['STD_LOGIC_VECTOR'] = ['A','B'] # OK
dict_test['IN']['STD_LOGIC_num_inputs'] = ['C','D'] # OK
dict_test['IN']['STD_LOGIC_VECTOR_num_inputs'] = ['E','X'] # OK
# dict_test['IN']['SIGNED'] = ['E'] # OK

dict_test['IN']['SIGNED_num_inputs'] = ['x','w'] # OK
# ----------------------------
dict_test['OUT']['STD_LOGIC'] = ['F','G'] # OK
dict_test['OUT']['STD_LOGIC_VECTOR'] = ['G','H'] # OK
dict_test['OUT']['STD_LOGIC_num_inputs'] = ['I','J'] # OK
dict_test['OUT']['STD_LOGIC_VECTOR_num_inputs'] = ['K','L'] # OK
# dict_test['OUT']['SIGNED'] = ['K'] #OK
dict_test['OUT']['SIGNED_num_inputs'] = ['M','N'] # OK

dict_test

{'IN': {'STD_LOGIC': ['clk', 'rst'],
  'STD_LOGIC_VECTOR': ['A', 'B'],
  'STD_LOGIC_num_inputs': ['C', 'D'],
  'STD_LOGIC_VECTOR_num_inputs': ['E', 'X'],
  'SIGNED': ['bias'],
  'SIGNED_num_inputs': ['x', 'w']},
 'OUT': {'STD_LOGIC': ['F', 'G'],
  'STD_LOGIC_VECTOR': ['G', 'H'],
  'STD_LOGIC_num_inputs': ['I', 'J'],
  'STD_LOGIC_VECTOR_num_inputs': ['K', 'L'],
  'SIGNED': ['y'],
  'SIGNED_num_inputs': ['M', 'N']}}

In [12]:
# Neuron_IN_port_map, Neuron_OUT_port_map, _, __ = Neuron_port_map_IO_unique(neuron_dict_list= dict_test, i = 2,
#                           num_inputs = 3,
#                           ID_camada = 'c1')

## <font color=green>    **Test**    </font>

In [13]:
# EXEMPLOS:
# print(input_sequences('x',num_inputs,'string',1,1)) # (sequence_inputs,'string',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences(['x','w'] ,num_inputs,'list',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences('x' ,num_inputs,'string',1,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences('x' _id ,num_inputs, list_or_string, port_map, new_line)

# print(input_sequences('sx' ,num_inputs,'string',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences('sw' ,num_inputs,'string',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences('val_x' ,num_inputs,'string',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences('val_w' ,num_inputs,'string',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
print(input_sequences(['x','w'] ,num_inputs,'list',1,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line))
print(input_sequences(['x','w'] ,num_inputs,'list',1,1)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line))

['x1=>x1', 'x2=>x2', 'x3=>x3', 'w1=>w1', 'w2=>w2', 'w3=>w3']
x1=>x1, 
x2=>x2, 
x3=>x3, 
w1=>w1, 
w2=>w2, 
w3=>w3, 



In [14]:
lista_camada_outputs = [[],None]
lista_camada_outputs[0] = []

In [15]:
txt_port_map, lista_camada_IO = port_map_dict(
        neuron_dict=layer_dict_hidden['Neuron_arch']['IO'],
        ID_camada= 'c1',
        lista_camada_IO=lista_camada_outputs[0],
        IO_type='IN',
        port_type='SIGNED',
        num_inputs=1,  # colocar = 1 quando 'com_numero' = False
        com_numero=False  # True: 'x' se torna 'x1, x2, ...'
    )

In [16]:
lista_camada_IO

['', ['c1bias']]

In [17]:
Neuron_IO_dict

{'IN': {'STD_LOGIC': ['clk', 'rst'],
  'STD_LOGIC_VECTOR': ['A', 'B'],
  'STD_LOGIC_num_inputs': ['C', 'D'],
  'STD_LOGIC_VECTOR_num_inputs': ['E', 'X'],
  'SIGNED': ['bias'],
  'SIGNED_num_inputs': ['x', 'w']},
 'OUT': {'STD_LOGIC': ['F', 'G'],
  'STD_LOGIC_VECTOR': ['G', 'H'],
  'STD_LOGIC_num_inputs': ['I', 'J'],
  'STD_LOGIC_VECTOR_num_inputs': ['K', 'L'],
  'SIGNED': ['y'],
  'SIGNED_num_inputs': ['M', 'N']}}

In [18]:
# 
print(input_sequences_number_choice(sequence_id = dict_test['IN']['STD_LOGIC'],
                                    port_map_list = dict_test['IN']['STD_LOGIC'],
                                    port_map_is_str = False,
                                    num_inputs = 1,
                                    list_or_string = 'list',
                                    port_map = 1,
                                    new_line = 1, 
                                    com_numero = False))

('clk=> clk, \nrst=> rst, \n', ['clk', 'rst'])


In [19]:
ID_camada= ' c1_n1_'
lista = dict_test['IN']['STD_LOGIC_VECTOR']
lista_var = [ID_camada + s for s in lista]
# lista_var
print(input_sequences_number_choice(lista,lista_var,False,1,'list',1,1, False))

('A=>  c1_n1_A, \nB=>  c1_n1_B, \n', [' c1_n1_A', ' c1_n1_B'])


In [20]:
for i in dict_test.items():
  print(i)

('IN', {'STD_LOGIC': ['clk', 'rst'], 'STD_LOGIC_VECTOR': ['A', 'B'], 'STD_LOGIC_num_inputs': ['C', 'D'], 'STD_LOGIC_VECTOR_num_inputs': ['E', 'X'], 'SIGNED': ['bias'], 'SIGNED_num_inputs': ['x', 'w']})
('OUT', {'STD_LOGIC': ['F', 'G'], 'STD_LOGIC_VECTOR': ['G', 'H'], 'STD_LOGIC_num_inputs': ['I', 'J'], 'STD_LOGIC_VECTOR_num_inputs': ['K', 'L'], 'SIGNED': ['y'], 'SIGNED_num_inputs': ['M', 'N']})


In [21]:
ID_camada = ' c1_n1_'
lista = dict_test['IN']['STD_LOGIC_num_inputs']
lista_var = [ID_camada + s for s in lista]
print(input_sequences_number_choice(lista,lista_var,False,num_inputs,'list',1,1, True))

('C1=>  c1_n1_C1, \nC2=>  c1_n1_C2, \nC3=>  c1_n1_C3, \nD1=>  c1_n1_D1, \nD2=>  c1_n1_D2, \nD3=>  c1_n1_D3, \n', [' c1_n1_C1', ' c1_n1_C2', ' c1_n1_C3', ' c1_n1_D1', ' c1_n1_D2', ' c1_n1_D3'])


In [22]:
ID_camada = ' c1_n1_'
lista = dict_test['IN']['STD_LOGIC_VECTOR_num_inputs']
lista_var = [ID_camada + s for s in lista]
print(input_sequences_number_choice(lista,lista_var,False,num_inputs,'list',1,1, True))

('E1=>  c1_n1_E1, \nE2=>  c1_n1_E2, \nE3=>  c1_n1_E3, \nX1=>  c1_n1_X1, \nX2=>  c1_n1_X2, \nX3=>  c1_n1_X3, \n', [' c1_n1_E1', ' c1_n1_E2', ' c1_n1_E3', ' c1_n1_X1', ' c1_n1_X2', ' c1_n1_X3'])


In [23]:
# print(input_sequences(dict_test['IN']['SIGNED_num_inputs'][0],num_inputs,'list',1,1))
print(input_sequences_number_choice(dict_test['IN']['SIGNED_num_inputs'][0],dict_test['IN']['SIGNED_num_inputs'][0],False,num_inputs,'list',1,1, True))

('x1=> x1, \nx2=> x2, \nx3=> x3, \n', ['x1', 'x2', 'x3'])


In [24]:
ID_camada= ' c1_n1_'        #string para adicionar no inicio de cada elemento da lista
lista = dict_test['IN']['SIGNED_num_inputs'][1]  # pega lista do dicionario (elementos wi)
lista_var = [ID_camada + s for s in lista] # unindo var_vamada com cada elemento da lista

print(input_sequences_number_choice(lista,lista_var,False,num_inputs,'list',1,1, True))

('w1=>  c1_n1_w1, \nw2=>  c1_n1_w2, \nw3=>  c1_n1_w3, \n', [' c1_n1_w1', ' c1_n1_w2', ' c1_n1_w3'])


# port_map_layer_ALL

In [25]:
print(port_map_dict(
        neuron_dict=layer_dict_hidden['Neuron_arch']['IO'],
        ID_camada=ID_camada,
        lista_camada_IO=[],
        IO_type='IN',
        port_type='STD_LOGIC'))

('clk1=> clk1, \nrst1=> rst1, \n', [['clk1', 'rst1'], ''])


Função geradora de mapeamento dos neurons da camada

In [26]:
port_map_txt, layer_dict_hidden = layer_neurons_port_map_ALL(
  layer_dict_arg = layer_dict_hidden,
  ID_camada = 'c1')

#layer_dict_hidden


In [27]:
layer_dict_hidden['IO']

{'IN': {'STD_LOGIC': ['clk', 'rst'],
  'STD_LOGIC_VECTOR': None,
  'SIGNED': ['c1_n0_bias',
   'c1_n1_bias',
   'c1_n2_bias',
   'c1_n3_bias',
   'x1',
   'x2',
   'x3',
   'c1_n0_w1',
   'c1_n0_w2',
   'c1_n0_w3',
   'c1_n1_w1',
   'c1_n1_w2',
   'c1_n1_w3',
   'c1_n2_w1',
   'c1_n2_w2',
   'c1_n2_w3',
   'c1_n3_w1',
   'c1_n3_w2',
   'c1_n3_w3']},
 'OUT': {'STD_LOGIC': None,
  'STD_LOGIC_VECTOR': None,
  'SIGNED': ['c1_n0_y', 'c1_n1_y', 'c1_n2_y', 'c1_n3_y']}}

# .vhd Names - ok

### <font color=green>    **.vhd Names > Sum (MAC)**    </font>

#### Name

In [28]:
IO_type

'signed'

In [29]:
sum_name = vhd_name('MAC', bits, IO_type, num_inputs, Barriers, MAC_type, Include_MAC_type)
if (DEBUG == 1):
  print(sum_name)

MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0


### <font color=green>    **.vhd Names > top_neuron**   </font>

#### <font color=orange>    **.vhd Names > top_neuron> Name**   </font>

In [30]:
prefix_neuron = 'neuron_'
fx_activation_dict = {
  'ReLU' : f'{prefix_neuron}ReLU',
  'Leaky ReLU': f'{prefix_neuron}LeakyReLU',
  'Sigmoid' : f'{prefix_neuron}Sigmoid'
}

In [31]:
# aqui estamos gerando o nome para cada um dos 3 tipos de neurônio e salvando em variáveis globais
neuron_ReLU_name = vhd_name(fx_activation_dict['ReLU'], bits, IO_type, num_inputs, Barriers, MAC_type, Include_MAC_type  )
neuron_Leaky_name = vhd_name(fx_activation_dict['Leaky ReLU'], bits, IO_type, num_inputs, Barriers, MAC_type, Include_MAC_type  )
neuron_Sigmoid_name = vhd_name(fx_activation_dict['Sigmoid'], bits, IO_type, num_inputs, Barriers, MAC_type, Include_MAC_type  )

if (DEBUG == 1):
  print(neuron_ReLU_name)
  print(neuron_Leaky_name)
  print(neuron_Sigmoid_name)

neuron_comb_barriers_ReLU_3n_8bit_unsigned_mul0_add0_v0_v0
neuron_comb_barriers_LeakyReLU_3n_8bit_unsigned_mul0_add0_v0_v0
neuron_comb_barriers_Sigmoid_3n_8bit_unsigned_mul0_add0_v0_v0


### <font color=green>    **.vhd Names > Testbench top neuron**   </font>

#### <font color=orange>    **.vhd Names > Testbench top neuron > Name**   </font>

In [32]:
# aqui estamos GERANDO OS NOMES para os arquivos de TESTBENCH para cada um dos 3 tipos de neurônio
tb_neuron_ReLU_name = vhd_name('tb_neuron_ReLU', bits, IO_type, num_inputs, Barriers, MAC_type, Include_MAC_type  )
tb_neuron_Leaky_name = vhd_name('tb_neuron_LeakyReLU', bits, IO_type, num_inputs, Barriers, MAC_type, Include_MAC_type  )
tb_neuron_soft_name = vhd_name('tb_neuron_Softmax', bits, IO_type, num_inputs, Barriers, MAC_type, Include_MAC_type  )

if (DEBUG == 1):
  print(tb_neuron_ReLU_name)
  print(tb_neuron_Leaky_name)
  print(tb_neuron_soft_name)

tb_comb_barriers_neuron_ReLU_3n_8bit_mul0_add0_v0_v0_unsigned
tb_comb_barriers_neuron_LeakyReLU_3n_8bit_mul0_add0_v0_v0_unsigned
tb_comb_barriers_neuron_Softmax_3n_8bit_mul0_add0_v0_v0_unsigned


# Neuronio

## <font color=green>**Sequências**</font>

In [33]:
# print(input_sequences('x' ,num_inputs,'string',1,1)) #(sequence_id ,num_inputs, list_or_string, port_map, new_line)

In [34]:
# print(input_sequences('x' ,
#                       num_inputs= 2,
#                       list_or_string = 'string',
#                       port_map = 0, 
#                       new_line= False))

In [35]:
# print(seq_input_output(x_seq = ['x1,x2,x3','w1,w2'],
#                        type = "IN",
#                        IO_type = 'signed',
#                        bits = 8,
#                        list_or_string = 'string',
#                        port_position = 'begin'))

In [36]:
# print(seq_input_output(x_seq = ['x1,x2,x3','w1,w2'],
#                        type = "IN",
#                        IO_type = 'signed',
#                        bits = 8,
#                        list_or_string = 'list',
#                        port_position = 'begin'))

In [37]:
# print(seq_signal(x_seq = ['x1,x2,x3','w1,w2'],
#                   type = "SIGNAL",
#                   IO_type = 'signed',
#                   bits = 8))

In [38]:
# inputs_vector(bits = 8,
#               IO_type = 'signed', 
#               x_seq= ['x1, x2, x3'], 
#               sx_seq = ['sx1, sx2, sx3'], 
#               valx_seq= ['val_x1, val_x2, val_x3'],
#               is_list= True
#               )

In [39]:
# inputs_vector(bits = 8,
#               IO_type = 'signed', 
#               x_seq= ['x1, x2, x3','w1, w2, w3'], 
#               sx_seq = ['sx1, sx2, sx3','sw1, sw2, sw3'], 
#               valx_seq= ['val_x1, val_x2, val_x3','val_w1, val_w2, val_w3'],
#               is_list= True
#               )

In [40]:
# inputs_vector(bits = 8,
#               IO_type = 'signed', 
#               x_seq= ['x1, x2, x3','w1, w2, w3'], 
#               sx_seq = ['sx1, sx2, sx3','sw1, sw2, sw3'], 
#               valx_seq= ['val_x1, val_x2, val_x3','val_w1, val_w2, val_w3'],
#               is_list= False
#               )

In [41]:
sequence_string_to_input(['x'], 3)

(['x1, x2, x3'],
 ['sx1, sx2, sx3'],
 ['val_x1, val_x2, val_x3'],
 ['x1', 'x2', 'x3'],
 ['x1=>x1', 'x2=>x2', 'x3=>x3'])

In [42]:
x,w,sx,sw,sx_tb,sw_tb,val_x,val_w,val_x_tb,val_w_tb = all_inputs_signals(['x','w'],num_inputs,bits,IO_type,0)
print(x)
print(w)
print(sx)
print(sw)
print(sx_tb)
print(sw_tb)
print(val_x)
print(val_w)
print(val_x_tb)
print(val_w_tb)

x1, x2, x3: IN signed(7 DOWNTO 0);
w1, w2, w3: IN signed(7 DOWNTO 0);
SIGNAL sx1, sx2, sx3: signed(7 DOWNTO 0); 
SIGNAL sw1, sw2, sw3: signed(7 DOWNTO 0); 
SIGNAL x1, x2, x3: signed(7 DOWNTO 0); 
SIGNAL w1, w2, w3: signed(7 DOWNTO 0); 
SIGNAL val_x1, val_x2, val_x3: STD_LOGIC_VECTOR(7 DOWNTO 0); 
SIGNAL val_w1, val_w2, val_w3: STD_LOGIC_VECTOR(7 DOWNTO 0); 
VARIABLE val_x1, val_x2, val_x3: STD_LOGIC_VECTOR(7 DOWNTO 0); 
VARIABLE val_w1, val_w2, val_w3: STD_LOGIC_VECTOR(7 DOWNTO 0); 


## <font color=red>**ROM**</font>

### <font color=green>    **.vhd Names > ROM**    </font>

In [43]:
# ROM_name = f"ROM_fx_{2**bits}rows_{bits}bitaddr_{bits}width"
ROM_name = f"ROM_fx_{bits}bitaddr_{bits}width"

if (DEBUG == 1):
  print(ROM_name)

ROM_fx_8bitaddr_8width


### <font color=orange>**ROM > COMPONENT**</font>

In [44]:
input_mem_bits

8

In [45]:
ROM_component = rom_component(ROM_name,input_mem_bits, output_mem_bits)
print(ROM_component)


  -- ROM 
  COMPONENT ROM_fx_8bitaddr_8width IS
    PORT ( 
      address : IN STD_LOGIC_VECTOR (7 DOWNTO 0);
      ------------------------------------------
      data_out : OUT STD_LOGIC_VECTOR (7  DOWNTO 0)
    );
  -- input: address (8 bits)	
  -- output: data_out (8 bits)
  END COMPONENT;                     
  


### <font color=orange>**ROM > PORT MAP**</font>

In [46]:
PORT_MAP_ROM = port_map_ROM(ROM_name,input_mem_bits,output_mem_bits)
print(PORT_MAP_ROM)


	U_ROM : ROM_fx_8bitaddr_8width PORT MAP(
		STD_LOGIC_VECTOR(out_reg_MAC), out_ROM_act
		);
	-- input: address (8)
	-- output: data_out (8)  
	


## <font color=red>**Sum (MAC)**</font>

### <font color=orange> **Sum (MAC)  > tree -> Combinacional (tree: árvore)** </font>

In [47]:
# spacebar_1  = "  "
# rst_space   = "      " #rst_receive (quando rst = '1')
# clk_space   = "        " #quando clk = '1'


In [48]:

spacebar_1  = 1
rst_space   = 3 #rst_receive (quando rst = '1')
clk_space   = 4 #quando clk = '1'

#### <font color=yellow> **Sum (MAC) > tree > Sequencias** </font>

In [49]:
import math
from math import log

log(num_inputs,2)

1.5849625007211563

In [50]:
# if (activ_fx_middle == 0): #_____________________COM LOOK UP TABLE________________________#
#   output_sum = ('''output: out STD_LOGIC_VECTOR(bits-1 DOWNTO 0) --output signal result \n ''')
#   sum_all_receive = ('sum_all<= signed(zero_middle_memory);\n')
# else:#______________________________SEM   MEMÓRIA____________________________________________#
#   output_sum = ('''output: out signed(bits-1 DOWNTO 0) --output signal result \n''')
#   sum_all_receive = ("sum_all<= (OTHERS => '0');\n")

output_sum = seq_input_output(['output'],'OUT',IO_type,bits,'string','end')
output_sum

'output: OUT signed(7 DOWNTO 0)'

##### <font color=#66FF00	> **Sum (MAC) > tree > Sequencias > Árvore de soma** </font>

In [51]:
signal_receive_string = signal_receive_input(num_inputs,1)
print(signal_receive_string)

  sx1 <= x1;
  sx2 <= x2;
  sx3 <= x3;
  sw1 <= w1;
  sw2 <= w2;
  sw3 <= w3;


In [52]:
# rst_receive_string = signal_receive_rst(rst_space, ['soma_1', 'soma_2', 'soma_3'],['mult_1', 'mult_2', 'mult_3'])
# print(rst_receivea_string)

In [53]:
# clk_receive_string, clk_receive = clk_receive_logic(clk_space,IO_type,bits,n_bin,mult_sequence)
# print(clk_receive_string)
# print(clk_receive)   

In [54]:
clk_receive_string, clk_receive = clk_receive_logic(
  clk_space = 4,
  IO_type = 'signed',
  bits = 8,
  n_bin = 8,
  sum_sequence= ['soma_1', 'soma_2', 'soma_3'],
  mult_sequence = ['mult_1', 'mult_2', 'mult_3'])

print(clk_receive_string)
print(clk_receive)   

        y <= signed(soma_3(15 DOWNTO 8));
        mult_1 <= (signed(sx1) * signed(sw1));
        mult_2 <= (signed(sx2) * signed(sw2));
        mult_3 <= (signed(sx3) * signed(sw3));
['        y <= signed(soma_3(15 DOWNTO 8));', '        mult_1 <= (signed(sx1) * signed(sw1));', '        mult_2 <= (signed(sx2) * signed(sw2));', '        mult_3 <= (signed(sx3) * signed(sw3));']


In [55]:
# sum_receive = tree_sum_algorithm(num_inputs,clk_space,sum_sequence,mult_sequence)

# sum_receive

In [59]:
s_sum,s_mult,signal_receive_string_MAC,rst_receive_string_MAC,clk_receive_string_MAC = arvore_de_soma_strings(num_inputs,bits,n_bin,IO_type,rst_space,clk_space)

if (DEBUG == 1):
  print(rst_receive_string_MAC)
  print('------------------------')
  print(clk_receive_string_MAC)

      soma_1 <= (OTHERS => '0');
      soma_2 <= (OTHERS => '0');
      soma_3 <= (OTHERS => '0');
      mult_1 <= (OTHERS => '0');
      mult_2 <= (OTHERS => '0');
      mult_3 <= (OTHERS => '0');
------------------------
        y <= signed(soma_3(15 DOWNTO 8));
        mult_1 <= (signed(sx1) * signed(sw1));
        mult_2 <= (signed(sx2) * signed(sw2));
        mult_3 <= (signed(sx3) * signed(sw3));
        soma_1 <=  mult_1 + mult_2;
        soma_2 <=  soma_1 + mult_3;
        soma_3 <= soma_2 + sbias; 




In [60]:
# print(s_sum)
# print("--")
# print(s_mult)
# print("--")
# print(signal_receive_string_MAC)

#### <font color=yellow> **Sum (MAC) > tree > Texto** </font>


###### <font color=#66FF00	> **Sum (MAC) > tree > Texto > Com barreiras temporais** </font>

In [61]:
TXT_MAC = ''
if (Barriers == 1): # COM BARREIRA
	TXT_MAC = (f'''
LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;
--use work.STD_DT.ALL;

ENTITY  {sum_name} IS
	--generic (input_bit:integer:= {str(bits -1)}; output_bit:integer:= {str(bits -1)};	 n_input:integer:={str(num_inputs)}); 
	PORT (
	clk, rst : IN STD_LOGIC;
	{x}
	{w}
	bias: IN {IO_type}({str(bits -1)} DOWNTO 0) ; -- offset
	------------------------------------------
	{output_sum}
);
-- clk, rst, 
-- inputs, 
-- weigths, 
-- bias, 
-- output
end ENTITY; 


ARCHITECTURE behavior of {sum_name} is
	SIGNAL sbias : {IO_type}({str(bits -1)} DOWNTO 0);
	SIGNAL sum_all : signed( {str((2*bits) -1)} DOWNTO 0);
	{sx}
	{sw}
	{s_sum}
	{s_mult}

BEGIN
{signal_receive_string_MAC}
	sbias <= bias;

	--sum_all <= ((signed(sx1)*signed(sw1))+ (signed(sx2)*signed(sw2)) + (signed(sx3)*signed(sw3))+ (signed(sx4)*signed(sw4)) + signed(bias));

	PROCESS (rst, clk)
	BEGIN
		IF (rst = '1') THEN 

{rst_receive_string_MAC}

		ELSE
			IF (clk'event AND clk = '1') THEN --se tem evento de clock 

{clk_receive_string_MAC}

			END IF;
		END IF;
	END PROCESS;

END behavior;''')
# init_text = 0
# print(TXT_MAC[init_text : (init_text+2000) ]) #Separa em linhas
# print(TXT_MAC[init_text: ]) #Separa em linhas
if (DEBUG == 1):
	print(TXT_MAC)


LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;
--use work.STD_DT.ALL;

ENTITY  MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
	--generic (input_bit:integer:= 7; output_bit:integer:= 7;	 n_input:integer:=3); 
	PORT (
	clk, rst : IN STD_LOGIC;
	x1, x2, x3: IN signed(7 DOWNTO 0);
	w1, w2, w3: IN signed(7 DOWNTO 0);
	bias: IN signed(7 DOWNTO 0) ; -- offset
	------------------------------------------
	output: OUT signed(7 DOWNTO 0)
);
-- clk, rst, 
-- inputs, 
-- weigths, 
-- bias, 
-- output
end ENTITY; 


ARCHITECTURE behavior of MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 is
	SIGNAL sbias : signed(7 DOWNTO 0);
	SIGNAL sum_all : signed( 15 DOWNTO 0);
	SIGNAL sx1, sx2, sx3: signed(7 DOWNTO 0); 
	SIGNAL sw1, sw2, sw3: signed(7 DOWNTO 0); 
	SIGNAL soma_1, soma_2, soma_3: signed(15 DOWNTO 0); 
	SIGNAL mult_1, mult_2, mult_3: signed(15 DOWNTO 0); 

BEGIN
  sx1 <= x1;
  sx2 <= x2;
  sx3 <= x3;
  sw1 <=

#### <font color=yellow> **IO Functions | MAC dict** </font>


In [62]:
MAC_IO_dict = {
  
  'IN': {
  # INPUTS
  'STD_LOGIC': ['clk', 'rst'] , #
  'STD_LOGIC_VECTOR': None, #['A','B'],
  # 'STD_LOGIC_VECTOR': ['A','B'],
  'STD_LOGIC_num_inputs': None, #['C','D'],
  'STD_LOGIC_VECTOR_num_inputs': None,
  'SIGNED': ['bias'],
  'SIGNED_num_inputs': ['x','w']
  },
  
  'OUT' : {
  #  OUTPUTS
  'STD_LOGIC': None , #
  'STD_LOGIC_VECTOR': None, #['A','B'],
  # 'STD_LOGIC_num_inputs': None, #['C','D'],
  'STD_LOGIC_num_inputs': None, #['C','D'],
  'STD_LOGIC_VECTOR_num_inputs': None,
  'SIGNED': ['output'],
  'SIGNED_num_inputs': None
    
  }
  }

if (DEBUG == 1):
  MAC_IO_dict

In [63]:
# Exemplo
if (DEBUG == 1):
  print(IO_STDL(MAC_IO_dict,1,'IN'))

clk, rst: IN STD_LOGIC;


In [64]:
if (DEBUG == 1):
  print(IO_signed(MAC_IO_dict,bits,1,'IN'))

bias: IN signed(7 DOWNTO 0);


In [65]:
# MAC_IO_dict['OUT']['STD_LOGIC_num_inputs'] = ['C','D']

In [66]:
MAC_IO_dict

{'IN': {'STD_LOGIC': ['clk', 'rst'],
  'STD_LOGIC_VECTOR': None,
  'STD_LOGIC_num_inputs': None,
  'STD_LOGIC_VECTOR_num_inputs': None,
  'SIGNED': ['bias'],
  'SIGNED_num_inputs': ['x', 'w']},
 'OUT': {'STD_LOGIC': None,
  'STD_LOGIC_VECTOR': None,
  'STD_LOGIC_num_inputs': None,
  'STD_LOGIC_VECTOR_num_inputs': None,
  'SIGNED': ['output'],
  'SIGNED_num_inputs': None}}

In [67]:
if (DEBUG == 1):
  print(IO_STD_num_inputs(MAC_IO_dict,num_inputs,0,'OUT'))

In [68]:
# x_sequence_string, sx_sequence_string, valx_sequence_string,x_w_sequence_string, xw_map_tb_sequence_string =sequence_string_to_input(['x','w'], num_inputs)
# print(x_sequence_string)

In [69]:
if (DEBUG == 1):
  print(IO_STDL_V_num_inputs(MAC_IO_dict,bits,num_inputs,0,'IN'))

In [70]:
if (DEBUG == 1):
  print(IO_signed_num_inputs(MAC_IO_dict,bits,num_inputs,0,'IN'))

x1, x2, x3: IN signed(7 DOWNTO 0);
w1, w2, w3: IN signed(7 DOWNTO 0);


In [71]:
# def IO_manager(IO_dict,bits,num_inputs,onerow,tab_space):
#   from txt_utils import erase_empty_lines, tab_lines
  
#   # INPUTS
#   IN_stdl = IO_STDL(IO_dict,onerow,'IN')
#   IN_stdl_v = IO_STDL_V(IO_dict,bits,onerow,'IN')
#   IN_stdl_num_inputs = IO_STD_num_inputs(IO_dict,num_inputs,onerow,'IN')
#   IN_stdl_v_num_inputs = IO_STDL_V_num_inputs(IO_dict,bits,num_inputs,0,'IN')
#   IN_signed = IO_signed(IO_dict,bits,onerow,'IN')
#   IN_signed_num_inputs = IO_signed_num_inputs(IO_dict,bits,num_inputs,0,'IN')
  
#   # OUTPUTS
#   OUT_stdl = IO_STDL(IO_dict,onerow,'OUT')
#   OUT_stdl_v = IO_STDL_V(IO_dict,bits,onerow,'OUT')
#   OUT_stdl_num_inputs = IO_STD_num_inputs(IO_dict,num_inputs,onerow,'OUT')
#   OUT_stdl_v_num_inputs = IO_STDL_V_num_inputs(IO_dict,bits,num_inputs,0,'OUT')
#   OUT_signed = IO_signed(IO_dict,bits,onerow,'OUT')
#   OUT_signed_num_inputs = IO_signed_num_inputs(IO_dict,bits,num_inputs,0,'OUT')
  
#   text = [IN_stdl,IN_stdl_v,IN_stdl_num_inputs,IN_signed,IN_signed_num_inputs,IN_stdl_v_num_inputs,
#           '----------------------------------------------',
#           OUT_stdl,OUT_stdl_v,OUT_stdl_num_inputs,OUT_signed,OUT_signed_num_inputs,OUT_stdl_v_num_inputs]
#   text = '\n'.join(map(str, (text))) #text_list can be an splitted text or a list of texts
#   traço = ' --'+'\n --'.join(map(str, (IO_dict.items())))
  
#   text = tab_lines(text, tab_space)
#   text = erase_empty_lines(text)[:-1]
#   # text = text[:-1] #tira ';' e uma linha em branco
#   return(text,traço)

# IO, traço = IO_manager(MAC_IO_dict,bits,num_inputs,1,2)

# if (DEBUG == 1):
#   print(IO)
#   # print(traço)

In [72]:
IO, traço = IO_manager([MAC_IO_dict],bits,num_inputs,1,2)

if (DEBUG == 1):
  print(IO)
  # print(traço)

    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    output: OUT signed(7 DOWNTO 0)


In [73]:
print(entity(sum_name, bits, num_inputs,[MAC_IO_dict]))

ENTITY  MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    output: OUT signed(7 DOWNTO 0)
    );
end ENTITY;


#### <font color=yellow> **Txt COM BARREIRA** </font>


In [74]:
MAC_signals = (f'''
        SIGNAL sbias : {IO_type}({str(bits -1)} DOWNTO 0);
	SIGNAL sum_all : signed( {str((2*bits) -1)} DOWNTO 0);
	{sx}
	{sw}
	{s_sum}
	{s_mult}''')
print(MAC_signals)


        SIGNAL sbias : signed(7 DOWNTO 0);
	SIGNAL sum_all : signed( 15 DOWNTO 0);
	SIGNAL sx1, sx2, sx3: signed(7 DOWNTO 0); 
	SIGNAL sw1, sw2, sw3: signed(7 DOWNTO 0); 
	SIGNAL soma_1, soma_2, soma_3: signed(15 DOWNTO 0); 
	SIGNAL mult_1, mult_2, mult_3: signed(15 DOWNTO 0); 


In [75]:
print(entity(name = sum_name, 
            bits = bits, 
            num_inputs = num_inputs,
            IO_dict_list = [MAC_IO_dict]))

ENTITY  MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    output: OUT signed(7 DOWNTO 0)
    );
end ENTITY;


In [76]:
TXT_MAC = ''
if (Barriers == 1): # COM BARREIRA
	TXT_MAC = (f'''
LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;
--use work.STD_DT.ALL;

{entity(sum_name, bits, num_inputs,[MAC_IO_dict])}


ARCHITECTURE behavior of {sum_name} is
{MAC_signals}

BEGIN
{signal_receive_string_MAC}
	sbias <= bias;

	--sum_all <= ((signed(sx1)*signed(sw1))+ (signed(sx2)*signed(sw2)) + (signed(sx3)*signed(sw3))+ (signed(sx4)*signed(sw4)) + signed(bias));

	PROCESS (rst, clk)
	BEGIN
		IF (rst = '1') THEN 

{rst_receive_string_MAC}

		ELSE
			IF (clk'event AND clk = '1') THEN --se tem evento de clock 

{clk_receive_string_MAC}

			END IF;
		END IF;
	END PROCESS;

END behavior;'''
	)
  
  

# init_text = 0
# print(TXT_MAC[init_text : (init_text+2000) ]) #Separa em linhas
# print(TXT_MAC[init_text: ]) #Separa em linhas
if (DEBUG == 1):
	print(TXT_MAC)


LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;
--use work.STD_DT.ALL;

ENTITY  MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    output: OUT signed(7 DOWNTO 0)
    );
end ENTITY;


ARCHITECTURE behavior of MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 is

        SIGNAL sbias : signed(7 DOWNTO 0);
	SIGNAL sum_all : signed( 15 DOWNTO 0);
	SIGNAL sx1, sx2, sx3: signed(7 DOWNTO 0); 
	SIGNAL sw1, sw2, sw3: signed(7 DOWNTO 0); 
	SIGNAL soma_1, soma_2, soma_3: signed(15 DOWNTO 0); 
	SIGNAL mult_1, mult_2, mult_3: signed(15 DOWNTO 0); 

BEGIN
  sx1 <= x1;
  sx2 <= x2;
  sx3 <= x3;
  sw1 <= w1;
  sw2 <= w2;
  sw3 <= w3;
	sbias <= bias;

	--sum_all <= ((signed(sx1)*signed(sw1))+ (signed(sx2)*signed(sw2)) + (signed(sx3)*signed(sw3))+ (signed

In [77]:
# TXT_MAC = entity(sum_name, bits, num_inputs,MAC_IO_dict)

###### <font color=#66FF00	> **Sum (MAC) > tree > Texto > Sem barreiras** </font>

In [78]:
#SOMATÓRIO -> operação de soma ponderada no sum.vhd
sum_all = []
i= 0

while True:
  if (i == 0):
    sum_all.append("sum_all <= (("+std_logic_conversion+"(sx"+str(i+1)+")*"+std_logic_conversion+"(sw"+str(i+1)+"))+")
    i= i+1
  
  else:
    if (i < num_inputs-1):
      sum_all.append("("+std_logic_conversion+"(sx"+str(i+1)+")*"+std_logic_conversion+"(sw"+str(i+1)+"))+")
      i= i+1
    else:
      sum_all.append("("+std_logic_conversion+"(sx"+str(i+1)+")*"+std_logic_conversion+"(sw"+str(i+1)+"))+"+std_logic_conversion+"(sbias));")
      i= 0
      break

sum_all_string = '\n'.join(map(str, sum_all)) 

if (DEBUG == 1):
  print(sum_all_string)

sum_all <= ((signed(sx1)*signed(sw1))+
(signed(sx2)*signed(sw2))+
(signed(sx3)*signed(sw3))+signed(sbias));


In [79]:
space = "  "
signal_receive = []

for i in signal_receive_string_MAC.splitlines():
  signal_receive.append(space + i)
  # print(space + i)
  
signal_receive = '\n'.join(map(str, signal_receive)) 
signal_receive_string = signal_receive

if (DEBUG == 1):
  print(signal_receive_string)

    sx1 <= x1;
    sx2 <= x2;
    sx3 <= x3;
    sw1 <= w1;
    sw2 <= w2;
    sw3 <= w3;


In [80]:
IO_type

'signed'

In [81]:
if IO_type == 'signed':
    IO_type_str = 'signed'
else:
    IO_type_str = 'unsigned'

In [82]:
# print(entity(sum_name, bits, num_inputs,[MAC_IO_dict]))

In [83]:
if (Barriers == 0): # SEM BARREIRA
  TXT_MAC = (f'''
LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;
--use work.STD_DT.ALL;

{entity(sum_name, bits, num_inputs,[MAC_IO_dict])}

ARCHITECTURE behavior of {sum_name} is
	SIGNAL sbias : ''' + IO_type_str + '''(''' +str(bits -1)+ ''' DOWNTO 0);
	SIGNAL sum_all : signed('''+ str((2*bits) -1) + ''' DOWNTO 0);
	'''+sx+'''
	'''+sw+'''

BEGIN
''' ""+signal_receive_string+"" '''
	sbias <= bias;
 
	'''+sum_all_string+'''

	--sum_all <= ((signed(sx1) * signed(sw1)) + (signed(sx2) * signed(sw2)) + (signed(sx3) * signed(sw3)) + (signed(sx4) * signed(sw4)) + signed(bias));

	PROCESS (rst, clk)
	BEGIN
		IF (rst = '1') THEN
			output <= (OTHERS => '0');
		ELSE
			IF (clk'event AND clk = '1') THEN --se tem evento de clock 
				output <= sum_all('''+ str((2*n_bin) -1) + ''' DOWNTO ('''+ str(n_bin) + '''));

			END IF;
		END IF;
	END PROCESS;

END behavior;

''')
if (DEBUG == 1):
	print(TXT_MAC)


LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;
--use work.STD_DT.ALL;

ENTITY  MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    output: OUT signed(7 DOWNTO 0)
    );
end ENTITY;


ARCHITECTURE behavior of MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 is

        SIGNAL sbias : signed(7 DOWNTO 0);
	SIGNAL sum_all : signed( 15 DOWNTO 0);
	SIGNAL sx1, sx2, sx3: signed(7 DOWNTO 0); 
	SIGNAL sw1, sw2, sw3: signed(7 DOWNTO 0); 
	SIGNAL soma_1, soma_2, soma_3: signed(15 DOWNTO 0); 
	SIGNAL mult_1, mult_2, mult_3: signed(15 DOWNTO 0); 

BEGIN
  sx1 <= x1;
  sx2 <= x2;
  sx3 <= x3;
  sw1 <= w1;
  sw2 <= w2;
  sw3 <= w3;
	sbias <= bias;

	--sum_all <= ((signed(sx1)*signed(sw1))+ (signed(sx2)*signed(sw2)) + (signed(sx3)*signed(sw3))+ (signed

In [84]:
# if (Barriers == 0): # SEM BARREIRA
#   TXT_MAC = ('''
# LIBRARY ieee;
# USE ieee.std_logic_1164.ALL;
# USE ieee.std_logic_unsigned.ALL;
# USE ieee.numeric_std.ALL;
# USE ieee.math_real.ALL;
# --use work.STD_DT.ALL;

# ENTITY  '''+sum_name+''' IS
# 	--generic (input_bit:integer:=''' +str(bits -1)+ '''; output_bit:integer:=''' +str(bits -1)+ ''';	 n_input:integer:=''' +str(num_inputs)+ '''); 
# 	PORT (
# 	clk, rst : IN STD_LOGIC;
# 	'''+x+'''
# 	'''+w+'''
# 	bias: IN '''+IO_type+'''(''' +str(bits -1)+ ''' DOWNTO 0) ; -- offset
# 	------------------------------------------
# 	'''+output_sum+'''
# );
# -- clk, rst, 
# -- inputs, 
# -- weigths, 
# -- bias, 
# -- output
# end ENTITY; 

# ARCHITECTURE behavior of '''+sum_name+''' is
# 	SIGNAL sbias : ''' + IO_type + '''(''' +str(bits -1)+ ''' DOWNTO 0);
# 	SIGNAL sum_all : signed('''+ str((2*bits) -1) + ''' DOWNTO 0);
# 	'''+sx+'''
# 	'''+sw+'''

# BEGIN
# ''' ""+signal_receive_string+"" '''
# 	sbias <= bias;
 
# 	'''+sum_all_string+'''

# 	--sum_all <= ((signed(sx1) * signed(sw1)) + (signed(sx2) * signed(sw2)) + (signed(sx3) * signed(sw3)) + (signed(sx4) * signed(sw4)) + signed(bias));

# 	PROCESS (rst, clk)
# 	BEGIN
# 		IF (rst = '1') THEN
# 			output <= (OTHERS => '0');
# 		ELSE
# 			IF (clk'event AND clk = '1') THEN --se tem evento de clock 
# 				output <= sum_all('''+ str((2*n_bin) -1) + ''' DOWNTO ('''+ str(n_bin) + '''));

# 			END IF;
# 		END IF;
# 	END PROCESS;

# END behavior;

# ''')
# if (DEBUG == 1):
# 	print(TXT_MAC)

#### <font color=yellow> **Sum (MAC) > tree > Imprimindo texto** </font>

In [85]:
# init_text = 0
# print(TXT_MAC[init_text : (init_text+2000) ]) #Separa em linhas
# print(TXT_MAC[init_text: ]) #Separa em linhas
# print(TXT_MAC)

In [86]:
print(TXT_MAC)


LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;
--use work.STD_DT.ALL;

ENTITY  MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    output: OUT signed(7 DOWNTO 0)
    );
end ENTITY;


ARCHITECTURE behavior of MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 is

        SIGNAL sbias : signed(7 DOWNTO 0);
	SIGNAL sum_all : signed( 15 DOWNTO 0);
	SIGNAL sx1, sx2, sx3: signed(7 DOWNTO 0); 
	SIGNAL sw1, sw2, sw3: signed(7 DOWNTO 0); 
	SIGNAL soma_1, soma_2, soma_3: signed(15 DOWNTO 0); 
	SIGNAL mult_1, mult_2, mult_3: signed(15 DOWNTO 0); 

BEGIN
  sx1 <= x1;
  sx2 <= x2;
  sx3 <= x3;
  sw1 <= w1;
  sw2 <= w2;
  sw3 <= w3;
	sbias <= bias;

	--sum_all <= ((signed(sx1)*signed(sw1))+ (signed(sx2)*signed(sw2)) + (signed(sx3)*signed(sw3))+ (signed

In [87]:
# text_file = open(sum_name+".txt", "w")
# text_file.write(sum_text)
# text_file.close()

# text_file = open(sum_name+".vhd", "w")
# text_file.write(sum_text)
# text_file.close()

if (download_vhd == 1):   #---------------- VHD ----------------# 
  # ReLU 
  with open(f"{path_ReLU}/{sum_name}.vhd", "w") as writer :
    writer.write(TXT_MAC)

  # Leaky ReLU 
  with open(f"{path_LeakyReLU}/{sum_name}.vhd", "w") as writer :
    writer.write(TXT_MAC)
    
  # SOFTMAX
  with open(f"{path_soft}/{sum_name}.vhd", "w") as writer :
    writer.write(TXT_MAC)    
  
if (download_txt == 1): #---------------- TEXTO ----------------#
  # ReLU 
  with open(f"{path_ReLU}/{sum_name}.txt", "w") as writer :
    writer.write(TXT_MAC)

  # Leaky ReLU 
  with open(f"{path_LeakyReLU}/{sum_name}.txt", "w") as writer :
    writer.write(TXT_MAC)
  
  # SOFTMAX
  with open(f"{path_soft}/{sum_name}.txt", "w") as writer :
    writer.write(TXT_MAC)

#### <font color=yellow> **Sum (MAC) > tree > Component** </font>

In [88]:
MAC_component = (f'''
	COMPONENT {str(sum_name)} IS
	PORT (
		clk, rst : IN STD_LOGIC;
		{x}
 		{w}
		bias: IN {IO_type}({str(bits -1)} DOWNTO 0) ; -- offset 
		------------------------------------------ 
		{output_sum}
	); -- clk, rst, inputs, weigths, bias, output
	END COMPONENT;                            
''')
if (DEBUG == 1):
	print(MAC_component)


	COMPONENT MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
	PORT (
		clk, rst : IN STD_LOGIC;
		x1, x2, x3: IN signed(7 DOWNTO 0);
 		w1, w2, w3: IN signed(7 DOWNTO 0);
		bias: IN signed(7 DOWNTO 0) ; -- offset 
		------------------------------------------ 
		output: OUT signed(7 DOWNTO 0)
	); -- clk, rst, inputs, weigths, bias, output
	END COMPONENT;                            



#### <font color=yellow> **Sum (MAC) > PORT MAP** </font>

In [89]:
reg_x_sequence = []
reg_w_sequence = []

reg_x_sequence_string = []
reg_w_sequence_string = []

i= 0 

while True:
  if (i < num_inputs):   
    #reg_xi e reg_wi
    reg_x_sequence.append("reg_x"+str(i+1))
    reg_w_sequence.append("reg_w"+str(i+1))  
    print()
    i= i+1
    
  else:
    i= 0
    break  
  
reg_x_sequence_string   = ', '.join(map(str, (reg_x_sequence))) 
reg_w_sequence_string   = ', '.join(map(str, (reg_w_sequence))) 

reg_x_sequence_string = reg_x_sequence_string + ","
reg_w_sequence_string = reg_w_sequence_string + ","

  
#https://stackoverflow.com/questions/15478127/remove-final-character-from-string
##reg_xi e reg_wi

s_reg_x = "SIGNAL "+reg_x_sequence_string[:-1]+": "+IO_type_str+"("+str(bits -1)+" DOWNTO 0); "

s_reg_w = "SIGNAL "+reg_w_sequence_string[:-1]+": "+IO_type_str+"("+str(bits -1)+" DOWNTO 0); "

if (DEBUG == 1):
  print(reg_x_sequence_string)
  print(reg_w_sequence_string)
  print(s_reg_x)
  print(s_reg_w)




reg_x1, reg_x2, reg_x3,
reg_w1, reg_w2, reg_w3,
SIGNAL reg_x1, reg_x2, reg_x3: unsigned(7 DOWNTO 0); 
SIGNAL reg_w1, reg_w2, reg_w3: unsigned(7 DOWNTO 0); 


In [90]:
PORT_MAP_MAC = ('''
	-- MAC ja registra a saida \n'''
"	U_MAC : "+ str(sum_name) + " PORT MAP(" '''
		clk, rst, \n'''
'	    	' + reg_x_sequence_string + '\n'
' 	   	' + reg_w_sequence_string + ' \n'
"		reg_bias, \n"
"		out_reg_MAC);"
)
print(PORT_MAP_MAC)


	-- MAC ja registra a saida 
	U_MAC : MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 PORT MAP(
		clk, rst, 
	    	reg_x1, reg_x2, reg_x3,
 	   	reg_w1, reg_w2, reg_w3, 
		reg_bias, 
		out_reg_MAC);


#### <font color=yellow> **Sum (MAC) > Function** </font>

In [91]:
# função mac aqui

### <font color=orange> **Sum (MAC)  > seq -> Sequencial (multiciclo)** </font>

#### <font color=66FF00> **Basics** </font>

##### <font color=1ECE03> **Adder** </font>

In [92]:
name_adder = "adder_" + str(bits) + "bit_" + str(num_inputs) +"input"
name_adder

'adder_8bit_3input'

In [93]:
txt_adder = ('''
LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_signed.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;
'''
"ENTITY "+ name_adder + " is " '''
port(
		'''"a,b: IN signed("+ str(2*bits) + "-1 DOWNTO 0); --inputs"'''
		output: OUT signed('''+ str(2*bits) + '''-1 DOWNTO 0) --output result 
);
END ENTITY;
'''
"ARCHITECTURE behavior of "+ name_adder + " is \n"
"SIGNAL sa,sb: signed("+ str(2*bits) + "-1 DOWNTO 0); " '''

begin 

 sa<=(a);
 sb<=(b);

end behavior; 
             ''')
# print(txt_adder)

In [94]:
# print(txt_adder)

##### <font color=1ECE03> **Mult** </font>

In [95]:
name_mult = "mult_" + str(bits) + "bit_" + str(num_inputs) +"input"
name_mult

'mult_8bit_3input'

In [96]:
txt_mult = ('''
LIBRARY ieee;
USE ieee.numeric_std.ALL;
USE IEEE.STD_LOGIC_UNSIGNED.ALL;
USE ieee.std_logic_1164.ALL;
USE work.STD_DT.ALL;

ENTITY '''+ name_mult + ''' IS
	PORT (
		x, w : IN signed('''+ str(bits - 1) + ''' DOWNTO 0); --inputs
		output : OUT signed('''+ str(2*bits - 1) + ''' DOWNTO 0) -- mult_out
	); --x,w,resultado
END ENTITY;

ARCHITECTURE behavior OF '''+ name_mult + ''' IS
	SIGNAL sx, sw : signed('''+ str(bits - 1) + ''' DOWNTO 0); --SIGNAL 

BEGIN
	sx <= (x);
	sw <= (w);

	output <= ((signed(sx) * signed(sw)));

END behavior;
             ''')
print(txt_mult)


LIBRARY ieee;
USE ieee.numeric_std.ALL;
USE IEEE.STD_LOGIC_UNSIGNED.ALL;
USE ieee.std_logic_1164.ALL;
USE work.STD_DT.ALL;

ENTITY mult_8bit_3input IS
	PORT (
		x, w : IN signed(7 DOWNTO 0); --inputs
		output : OUT signed(15 DOWNTO 0) -- mult_out
	); --x,w,resultado
END ENTITY;

ARCHITECTURE behavior OF mult_8bit_3input IS
	SIGNAL sx, sw : signed(7 DOWNTO 0); --SIGNAL 

BEGIN
	sx <= (x);
	sw <= (w);

	output <= ((signed(sx) * signed(sw)));

END behavior;
             


In [97]:
# print(txt_mult)

##### <font color=1ECE03> **Mux 2 -> 1** </font>

In [98]:
name_mux_2_1 = "mux_" + str(bits) + "bit_" + str(num_inputs) +"input"
name_mux_2_1

'mux_8bit_3input'

In [99]:
txt_mux_2_1 = ('''
LIBRARY ieee;
USE ieee.numeric_std.ALL;
USE IEEE.STD_LOGIC_UNSIGNED.ALL;
USE ieee.std_logic_1164.ALL;
USE work.STD_DT.ALL;

ENTITY '''+ name_mux_2_1 + ''' IS
     PORT (
          a, b : IN signed('''+ str(bits -1) + ''' DOWNTO 0);
          sel : IN STD_LOGIC;
          y : OUT signed('''+ str(bits -1) + ''' DOWNTO 0)
     );--a,b,sel,output
END ENTITY;

ARCHITECTURE behavior OF '''+ name_mux_2_1 + ''' IS
BEGIN
     WITH sel SELECT
          y <= a WHEN '0',
          b WHEN OTHERS;
END behavior;               
               ''')
# print(txt_mux_2_1)

In [100]:
# print(txt_mux_2_1)

##### <font color=1ECE03> **Mux 2 -> 1 x2** </font>

In [101]:
name_mux_2_1_2x = "mux_2_1_" + str(bits) + "bit" #+ str(num_inputs) +"input"
name_mux_2_1_2x

'mux_2_1_8bit'

In [102]:
txt_mux_2_1_2x = ('''
LIBRARY ieee;
USE ieee.numeric_std.ALL;
USE IEEE.STD_LOGIC_UNSIGNED.ALL;
USE ieee.std_logic_1164.ALL;
USE work.STD_DT.ALL;

ENTITY '''+ name_mux_2_1_2x+ ''' IS
     PORT (
          a, b : IN signed(('''+ str(2*bits -1) + ''') DOWNTO 0);
          sel : IN STD_LOGIC;
          y : OUT signed(('''+ str(2*bits -1) + ''') DOWNTO 0)
     );--a,b,sel,output
END ENTITY;

ARCHITECTURE behavior OF '''+ name_mux_2_1_2x + ''' IS
BEGIN
     WITH sel SELECT
          y <= a WHEN '0',
          b WHEN OTHERS;
END behavior;               
               ''')
# print(txt_mux_2_1_2x)

In [103]:
# print(txt_mux_2_1_2x)

##### <font color=1ECE03> **Mux 4 -> 1** </font>

In [104]:
name_mux_4_1 = "mux_" + str(bits) + "bit_" + str(num_inputs) +"input"
name_mux_4_1

'mux_8bit_3input'

In [105]:
states_mux = math.ceil(log(num_inputs,2))
states_mux

2

In [106]:
txt_mux_4_1 = ('''
LIBRARY ieee;
USE ieee.numeric_std.ALL;
USE IEEE.STD_LOGIC_UNSIGNED.ALL;
USE ieee.std_logic_1164.ALL;
USE work.STD_DT.ALL;

ENTITY '''+ name_mult + ''' IS
	PORT (
		a, b, c, d : IN signed('''+ str(bits -1) + ''' DOWNTO 0);
		sel : IN STD_LOGIC_VECTOR('''+ str(states_mux -1) + ''' DOWNTO 0);
		y : OUT signed('''+ str(bits -1) + ''' DOWNTO 0)); --a,b,c,d,sel,y
END ENTITY;

ARCHITECTURE comportamento OF '''+ name_mult + ''' IS
	SIGNAL reg_y : signed('''+ str(bits -1) + ''' DOWNTO 0);
BEGIN
	--process(rst,clk,a,b,c,d,bias) 
	PROCESS (sel, a, b, c, d)
	BEGIN

		CASE sel IS
			WHEN "00" => --estado de rst
				reg_y <= a;
			WHEN "01" => --estado de rst
				reg_y <= b;
			WHEN "10" => --estado de rst
				reg_y <= c;
			WHEN "11" => --estado de rst
				reg_y <= d;

			WHEN OTHERS => reg_y <= (OTHERS => '0');

		END CASE;
	END PROCESS;

	y <= reg_y;
END comportamento;               
               ''')
# print(txt_mux_4_1)

In [107]:
# print(txt_mux_4_1)

##### NAME

In [108]:
name_NAME = "NAME_" + str(bits) + "bit_" + str(num_inputs) +"input"
name_NAME

'NAME_8bit_3input'

In [109]:
txt_NAME = ('''
               
               ''')
print(txt_NAME)

##### NAME

In [110]:
name_NAME = "NAME_" + str(bits) + "bit_" + str(num_inputs) +"input"
name_NAME

'NAME_8bit_3input'

In [111]:
txt_NAME = ('''
               
               ''')
print(txt_NAME)

##### NAME

In [112]:
name_NAME = "NAME_" + str(bits) + "bit_" + str(num_inputs) +"input"
name_NAME

'NAME_8bit_3input'

In [113]:
txt_NAME = ('''
               
               ''')
print(txt_NAME)

#### <font color=66FF00> **Sum (MAC)  > seq > Sequencias** </font>

##### Sum (MAC)  > seq > Texto

In [114]:
# MAC_seq_text = ('''LIBRARY ieee;
# USE ieee.std_logic_1164.ALL;
# USE ieee.std_logic_unsigned.ALL;
# USE ieee.numeric_std.ALL;
# USE ieee.math_real.ALL;
# --use work.STD_DT.ALL;
# '''
# "ENTITY  "+sum_name+" IS " '''
# 	--generic (input_bit:integer:=8; output_bit:integer:=8;	 n_input:integer:=3); 
# 	PORT (
# 	clk, rst  : IN STD_LOGIC;'''
# '	\n'
# '	'+x+''
# ' 	'+w+''
# "	bias: IN "+IO_type+"(" +str(bits -1)+ " DOWNTO 0) ; -- offset \n"
# "	------------------------------------------ \n"
# '	'+output_sum+''
# ");"
# ''' 
# -- clk, rst, 
# -- inputs, 
# -- weigths, 
# -- bias, 
# -- output
# end ENTITY; 

# '''
# "ARCHITECTURE behavior of "+sum_name+" is\n"
# "	SIGNAL sbias : signed(8 - 1 DOWNTO 0); "
# '	'+sx+''
# ' 	'+sw+' '
# ' '+s_sum+''
# ' 	'+s_mult+''
# '''
# BEGIN \n'''
# ""+signal_receive_string+""
# "\n	sbias <= bias; \n"
# '''
#  	--sum_all <= ((signed(sx1)*signed(sw1))+ (signed(sx2)*signed(sw2)) + (signed(sx3)*signed(sw3))+ (signed(sx4)*signed(sw4)) + signed(bias));

# 	PROCESS (rst, clk)
# 	BEGIN
# 		IF (rst = '1') THEN \n'''

# ""+rst_receive_string+" \n"
# '''
#  		ELSE
#  			IF (clk'event AND clk = '1') THEN --se tem evento de clock 
# '''
# ""+clk_receive_string+""

# 			# output <= (OTHERS => '0');

# 			# mult_1 <= (OTHERS => '0');
# 			# mult_2 <= (OTHERS => '0');
# 			# mult_3 <= (OTHERS => '0');
# 			# mult_4 <= (OTHERS => '0');

# 			# soma_1 <= (OTHERS => '0');
# 			# soma_2 <= (OTHERS => '0');
# 			# sum_all <= (OTHERS => '0');

# # 		ELSE
# # 			IF (clk'event AND clk = '1') THEN --se tem evento de clock 
# # 				output <= sum_all((11) - 1 DOWNTO (3));

# # 				mult_1 <= (signed(sx1) * signed(sw1));
# # 				mult_2 <= (signed(sx2) * signed(sw2));
# # 				mult_3 <= (signed(sx3) * signed(sw3));
# # 				mult_4 <= (signed(sx4) * signed(sw4));

# # 				soma_1 <= mult_1 + mult_2;
# # 				soma_2 <= mult_3 + mult_4;
# # 				sum_all <= soma_1 + soma_2;
# '''
# 			END IF;
# 		END IF;
# 	END PROCESS;

# END behavior;'''
# )

# init_text = 0
# print(MAC_seq_text[init_text : (init_text+2000) ]) #Separa em linhas
# # print(MAC_seq_text[init_text: ]) #Separa em linhas
# # print(MAC_seq_text)

##### Sum (MAC)  > seq > Imprimindo texto

##### Sum (MAC)  > seq > Component

##### Sum (MAC)  > seq > PORT MAP

## <font color=red>**Top Neuronio**</font>

In [115]:
rst_space = "      "    #rst_receive (quando rst = '1')
clk_space = "        "  #quando clk = '1'

### <font color=orange> **Top Neuronio > tree** </font>

#### <font color=yellow> **Top Neuronio > tree > Sequencias** </font>

In [116]:
num_inputs

3

In [117]:
# reg_x_sequence = []
# reg_w_sequence = []

# reg_x_sequence_string = []
# reg_w_sequence_string = []

# i= 0 

# while True:
#   if (i < num_inputs):   
#     #reg_xi e reg_wi
#     reg_x_sequence.append("reg_x"+str(i+1))
#     reg_w_sequence.append("reg_w"+str(i+1))  
#     print()
#     i= i+1
    
#   else:
#     i= 0
#     break  
  
# reg_x_sequence_string   = ', '.join(map(str, (reg_x_sequence))) 
# reg_w_sequence_string   = ', '.join(map(str, (reg_w_sequence))) 
  
# #https://stackoverflow.com/questions/15478127/remove-final-character-from-string
# ##reg_xi e reg_wi

# s_reg_x = "SIGNAL "+reg_x_sequence_string+": "+IO_type+"("+str(bits -1)+" DOWNTO 0); "

# s_reg_w = "SIGNAL "+reg_w_sequence_string+": "+IO_type+"("+str(bits -1)+" DOWNTO 0); "

# print(s_reg_x,s_reg_w)

In [118]:
rst_receive = []
rst_receive.append(rst_space + "reg_out_ROM_act <= (OTHERS => '0');")
rst_receive.append("")

for i in range(0,len(reg_x_sequence)):
  rst_receive.append(rst_space + reg_x_sequence[i]+" <= (OTHERS => '0');")

rst_receive.append("")

for i in  range(0,len(reg_w_sequence)):
  rst_receive.append(rst_space + reg_w_sequence[i]+" <= (OTHERS => '0');")

rst_receive_string = '\n'.join(map(str, (rst_receive))) 
rst_receive_string =  (rst_receive_string + "\n" + 
                         rst_space + "reg_bias <= (OTHERS => '0');")
# print(rst_receive_string)

In [119]:
x

'x1, x2, x3: IN signed(7 DOWNTO 0);'

In [120]:
x_sequence = input_sequences('x' ,num_inputs,'list',0,0)
x_sequence

['x1', 'x2', 'x3']

In [121]:
clk_receive = []
clk_receive.append( clk_space[:-2] + "reg_out_ROM_act <=  signed(out_ROM_act); ")
clk_receive.append( clk_space + "IF update_weights = '0' THEN ")


for i in range(0,len(reg_x_sequence)):
  clk_receive.append(clk_space + "  " + reg_x_sequence[i]+" <= " +  x_sequence[i]  + ";")

clk_receive.append("")
clk_receive.append( clk_space + "ELSE")


for i in  range(0,len(reg_w_sequence)):
  clk_receive.append(clk_space + "  " + reg_w_sequence[i]+" <= " +  x_sequence[i]  + ";")
clk_receive.append("")
  
clk_receive.append( clk_space + "END IF;")


clk_receive_string = '\n'.join(map(str, (clk_receive))) 
clk_receive_string =  (clk_receive_string + "\n" + 
                         clk_space + "reg_bias <= bias;")

print(clk_receive_string)

      reg_out_ROM_act <=  signed(out_ROM_act); 
        IF update_weights = '0' THEN 
          reg_x1 <= x1;
          reg_x2 <= x2;
          reg_x3 <= x3;

        ELSE
          reg_w1 <= x1;
          reg_w2 <= x2;
          reg_w3 <= x3;

        END IF;
        reg_bias <= bias;


In [122]:
# print(rst_receive_string)

#### <font color=yellow>**Top Neuronio > tree > NEURON_SOFT_INPUT_NET**</font>

In [123]:
NEURON_SOFT_INPUT_NET = ('''
	PROCESS (clk, rst, update_weights)
	BEGIN
		IF rst = '1' THEN \n'''

""+rst_receive_string+" \n"
'''
		ELSIF clk'event AND clk = '1' THEN
'''
""+clk_receive_string+""
'''

		END IF;
	END PROCESS;
''')
if (DEBUG == 1):
	print(NEURON_SOFT_INPUT_NET)


	PROCESS (clk, rst, update_weights)
	BEGIN
		IF rst = '1' THEN 
      reg_out_ROM_act <= (OTHERS => '0');

      reg_x1 <= (OTHERS => '0');
      reg_x2 <= (OTHERS => '0');
      reg_x3 <= (OTHERS => '0');

      reg_w1 <= (OTHERS => '0');
      reg_w2 <= (OTHERS => '0');
      reg_w3 <= (OTHERS => '0');
      reg_bias <= (OTHERS => '0'); 

		ELSIF clk'event AND clk = '1' THEN
      reg_out_ROM_act <=  signed(out_ROM_act); 
        IF update_weights = '0' THEN 
          reg_x1 <= x1;
          reg_x2 <= x2;
          reg_x3 <= x3;

        ELSE
          reg_w1 <= x1;
          reg_w2 <= x2;
          reg_w3 <= x3;

        END IF;
        reg_bias <= bias;

		END IF;
	END PROCESS;



#### <font color=yellow>**Top Neuronio > tree > NEURON_ReLU_INPUT_NET**</font>

In [124]:
rst_receive_string_middle = '\n'.join(rst_receive_string.splitlines()[2:])
# rst_receive_string_middle = rst_space + "y <= (OTHERS => '0'); \n" + rst_receive_string_middle

# print(rst_receive_string_middle)

In [125]:
clk_receive_string_middle = '\n'.join(clk_receive_string.splitlines()[1:])
# clk_receive_string_middle = clk_space + mux_LeakyReLU + clk_receive_string_middle
    
print(clk_receive_string_middle)    

        IF update_weights = '0' THEN 
          reg_x1 <= x1;
          reg_x2 <= x2;
          reg_x3 <= x3;

        ELSE
          reg_w1 <= x1;
          reg_w2 <= x2;
          reg_w3 <= x3;

        END IF;
        reg_bias <= bias;


In [126]:
mux_ReLU = ('''
			IF out_reg_MAC > 0 THEN
				y <= out_reg_MAC;
			ELSE
				y <= (OTHERS => '0');
			END IF;
''')
# print(mux_LeakyReLU)

In [127]:
NEURON_ReLU_INPUT_NET = ('''
	PROCESS (clk, rst, update_weights)
	BEGIN
		IF rst = '1' THEN \n'''

""+rst_receive_string_middle+" \n"
'''
		ELSIF clk'event AND clk = '1' THEN
'''+ mux_ReLU + 
""+clk_receive_string_middle+""
'''
		END IF;
	END PROCESS;
''')

print(NEURON_ReLU_INPUT_NET)


	PROCESS (clk, rst, update_weights)
	BEGIN
		IF rst = '1' THEN 
      reg_x1 <= (OTHERS => '0');
      reg_x2 <= (OTHERS => '0');
      reg_x3 <= (OTHERS => '0');

      reg_w1 <= (OTHERS => '0');
      reg_w2 <= (OTHERS => '0');
      reg_w3 <= (OTHERS => '0');
      reg_bias <= (OTHERS => '0'); 

		ELSIF clk'event AND clk = '1' THEN

			IF out_reg_MAC > 0 THEN
				y <= out_reg_MAC;
			ELSE
				y <= (OTHERS => '0');
			END IF;
        IF update_weights = '0' THEN 
          reg_x1 <= x1;
          reg_x2 <= x2;
          reg_x3 <= x3;

        ELSE
          reg_w1 <= x1;
          reg_w2 <= x2;
          reg_w3 <= x3;

        END IF;
        reg_bias <= bias;
		END IF;
	END PROCESS;



#### <font color=yellow>**Top Neuronio > tree > NEURON_LeakyReLU_INPUT_NET**</font>

In [128]:
ones_leaky = str(np.ones(leaky_attenuation))
ones_leaky= ones_leaky.replace(".", "");
ones_leaky= ones_leaky.replace("[", "");
ones_leaky= ones_leaky.replace("]", "");
ones_leaky= ones_leaky.replace(" ", "");
ones_leaky

'11'

In [129]:
mux_LeakyReLU = (f'''
			IF out_reg_MAC > 0 THEN
				y <= out_reg_MAC;
			ELSE
				y <=  {ones_leaky} & out_reg_MAC(
     {str(bits -1)}  DOWNTO {str(leaky_attenuation)});
			END IF;
''')
# print(mux_LeakyReLU)

In [130]:
NEURON_LeakyReLU_INPUT_NET = (f'''
	PROCESS (clk, rst, update_weights)
	BEGIN
		IF rst = '1' THEN \n

{rst_receive_string_middle}

		ELSIF clk'event AND clk = '1' THEN
 {mux_LeakyReLU} 
{clk_receive_string_middle}

		END IF;
	END PROCESS;
''')

print(NEURON_LeakyReLU_INPUT_NET)


	PROCESS (clk, rst, update_weights)
	BEGIN
		IF rst = '1' THEN 


      reg_x1 <= (OTHERS => '0');
      reg_x2 <= (OTHERS => '0');
      reg_x3 <= (OTHERS => '0');

      reg_w1 <= (OTHERS => '0');
      reg_w2 <= (OTHERS => '0');
      reg_w3 <= (OTHERS => '0');
      reg_bias <= (OTHERS => '0');

		ELSIF clk'event AND clk = '1' THEN
 
			IF out_reg_MAC > 0 THEN
				y <= out_reg_MAC;
			ELSE
				y <=  11 & out_reg_MAC(
     7  DOWNTO 2);
			END IF;
 
        IF update_weights = '0' THEN 
          reg_x1 <= x1;
          reg_x2 <= x2;
          reg_x3 <= x3;

        ELSE
          reg_w1 <= x1;
          reg_w2 <= x2;
          reg_w3 <= x3;

        END IF;
        reg_bias <= bias;

		END IF;
	END PROCESS;



#### <font color=yellow> **Top Neuronio > tree > Função neuronio (não foi feita)** </font>

In [131]:
# rst_space = "     " #rst_receive (quando rst = '1')
# clk_space = "      " #quando clk = '1'

# def Funcao_Neuronio (num_inputs,) {
  
#   #------------------------------ reg_xi e reg_wi ---------------------#
#   reg_x_sequence = []
#   reg_w_sequence = []

#   reg_x_sequence_string = []
#   reg_w_sequence_string = []

#   i= 0 

#   while True:
#     if (i < num_inputs):   
#       #reg_xi e reg_wi
#       reg_x_sequence.append("reg_x"+str(i+1))
#       reg_w_sequence.append("reg_w"+str(i+1))  
#       print()
#       i= i+1
      
#     else:
#       i= 0
#       break  
    
#   reg_x_sequence_string   = ', '.join(map(str, (reg_x_sequence))) 
#   reg_w_sequence_string   = ', '.join(map(str, (reg_w_sequence))) 
    
#   #https://stackoverflow.com/questions/15478127/remove-final-character-from-string
#   ##reg_xi e reg_wi

#   s_reg_x = "SIGNAL "+reg_x_sequence_string+": "+IO_type+"("+str(bits -1)+" DOWNTO 0); "

#   s_reg_w = "SIGNAL "+reg_w_sequence_string+": "+IO_type+"("+str(bits -1)+" DOWNTO 0); "
  
#   #------------------------------ rst Xi Wi SEQUENCE ---------------------#
#   rst_receive = []
#   rst_receive.append(rst_space + "reg_out_rom_act <= (OTHERS => '0');")
#   rst_receive.append("")

#   for i in range(0,len(reg_x_sequence)):
#     rst_receive.append(rst_space + reg_x_sequence[i]+" <= (OTHERS => '0');")

#   rst_receive.append("")

#   for i in  range(0,len(reg_w_sequence)):
#     rst_receive.append(rst_space + reg_w_sequence[i]+" <= (OTHERS => '0');")

#   rst_receive_string = '\n'.join(map(str, (rst_receive))) 
#   rst_receive_string =  (rst_receive_string + "\n" + 
#                           rst_space + "reg_bias <= (OTHERS => '0');")
#   #------------------------------ CLK Xi Wi SEQUENCE ---------------------#
  
#   clk_receive = []
#   clk_receive.append( clk_space + "reg_out_rom_act <=  signed(out_rom_act); ")
#   clk_receive.append("")

#   for i in range(0,len(reg_x_sequence)):
#     clk_receive.append(clk_space + reg_x_sequence[i]+" <= " +  x_sequence[i]  + ";")

#   clk_receive.append("")

#   for i in  range(0,len(reg_w_sequence)):
#     clk_receive.append(clk_space + reg_w_sequence[i]+" <= " +  x_sequence[i]  + ";")

#   clk_receive_string = '\n'.join(map(str, (clk_receive))) 
#   clk_receive_string =  (clk_receive_string + "\n" + 
#                           clk_space + "reg_bias <= bias;")

  
#   NEURON_SOFT_INPUT_NET = ('''
#     PROCESS (clk, rst)
#     BEGIN
#       IF rst = '1' THEN \n'''

#   ""+rst_receive_string+" \n"
#   '''
#       ELSIF clk'event AND clk = '1' THEN
#   '''
#   ""+clk_receive_string+""
#   '''

#         END IF;
#       END PROCESS;
#   ''')

  
#   return (NEURON_SOFT_INPUT_NET)
# }

#### <font color=yellow> **Top Neuronio > tree > Texto** </font>

##### <font color=cyan> **Top Neuronio > tree > Função texto** </font>

In [132]:
# layers_dict_list[0]

In [133]:
print(entity(name = layer_dict_hidden['Neuron_arch']['neuron_name'], 
             bits = 8, 
             num_inputs = 3,
             IO_dict_list = [layer_dict_hidden['Neuron_arch']['IO']['shared_IO'], layer_dict_hidden['Neuron_arch']['IO']['unique_IO']]
             ))


ENTITY  neuron_comb_ReLU_3n_8bit_signed_mul0_v0_add0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    y: OUT signed(7 DOWNTO 0)
    );
end ENTITY;


In [134]:
# layers_dict_list[0]['Neuron_arch']['Inputs_number']()

##### <font color=lime> **Top Neuronio > tree > Texto > Softmax** </font>

In [135]:
Neuron_IO_dict = {
  'IN': { # ENTRADAS
    'STD_LOGIC': ['clk', 'rst'],
    'STD_LOGIC_VECTOR': None,
    'STD_LOGIC_num_inputs': None,
    'STD_LOGIC_VECTOR_num_inputs': None,
    'SIGNED': ['bias'],
    'SIGNED_num_inputs': ['x']
    },
 'OUT': { #SAÍDAS
   'STD_LOGIC': None,
    'STD_LOGIC_VECTOR': None,
    'STD_LOGIC_num_inputs': None,
    'STD_LOGIC_VECTOR_num_inputs': None,
    'SIGNED': ['y'],
    'SIGNED_num_inputs': None
    }
 }
Neuron_IO_dict

{'IN': {'STD_LOGIC': ['clk', 'rst'],
  'STD_LOGIC_VECTOR': None,
  'STD_LOGIC_num_inputs': None,
  'STD_LOGIC_VECTOR_num_inputs': None,
  'SIGNED': ['bias'],
  'SIGNED_num_inputs': ['x']},
 'OUT': {'STD_LOGIC': None,
  'STD_LOGIC_VECTOR': None,
  'STD_LOGIC_num_inputs': None,
  'STD_LOGIC_VECTOR_num_inputs': None,
  'SIGNED': ['y'],
  'SIGNED_num_inputs': None}}

In [136]:
print(entity(neuron_Sigmoid_name, bits, num_inputs,[Neuron_IO_dict]))

ENTITY  neuron_comb_barriers_Sigmoid_3n_8bit_unsigned_mul0_add0_v0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    y: OUT signed(7 DOWNTO 0)
    );
end ENTITY;


In [137]:
top_neuron_soft_txt = (f'''LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;

{entity(neuron_Sigmoid_name, bits, num_inputs,[Neuron_IO_dict])}


ARCHITECTURE behavior of {neuron_Sigmoid_name} is
{MAC_component}
{ROM_component}

	SIGNAL out_reg_MAC : {IO_type} (( {str(bits - 1)} ) DOWNTO 0);	--reg da saida do MAC'''+
# "	SIGNAL out_reg_MAC : STD_LOGIC_VECTOR(("+ str(bits - 1) + ") DOWNTO 0);	--reg da saida do MAC	\n"
f'''
	SIGNAL out_ROM_act : STD_LOGIC_VECTOR( {str(bits - 1)} DOWNTO 0); --saida da ROM
	SIGNAL reg_out_ROM_act : {IO_type} ( {str(bits - 1)} DOWNTO 0); --reg saida da ROM

	{s_reg_x}
 	{s_reg_w}
	SIGNAL reg_bias : {IO_type} ( {str(bits - 1)} DOWNTO 0);


 BEGIN
	{PORT_MAP_MAC}
	{PORT_MAP_ROM}
	 {NEURON_SOFT_INPUT_NET}
  
 	y <= {IO_type} (reg_out_ROM_act);

END behavior;'''
)

# init_text = 1450
# print(top_neuron_soft_txt[init_text : (init_text+2000) ]) #Separa em linhas
# print(top_neuron_soft_txt[init_text: ]) #Separa em linhas
print(top_neuron_soft_txt)

LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;

ENTITY  neuron_comb_barriers_Sigmoid_3n_8bit_unsigned_mul0_add0_v0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    y: OUT signed(7 DOWNTO 0)
    );
end ENTITY;


ARCHITECTURE behavior of neuron_comb_barriers_Sigmoid_3n_8bit_unsigned_mul0_add0_v0_v0 is

	COMPONENT MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
	PORT (
		clk, rst : IN STD_LOGIC;
		x1, x2, x3: IN signed(7 DOWNTO 0);
 		w1, w2, w3: IN signed(7 DOWNTO 0);
		bias: IN signed(7 DOWNTO 0) ; -- offset 
		------------------------------------------ 
		output: OUT signed(7 DOWNTO 0)
	); -- clk, rst, inputs, weigths, bias, output
	END COMPONENT;                            


  -- ROM 
  COMPONENT ROM_fx_8bitaddr_8width IS
    PORT ( 
      address : IN STD_LOGIC_VECTOR (7 DOWNTO 0)

##### <font color=lime> **Top Neuronio > tree > Texto > Middle NN** </font>

###### <font color=00FFE6> **Top Neuronio > tree > Texto > Middle NN > ReLU** </font>

In [138]:
top_neuron_ReLU_txt = (f'''LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;

ENTITY  {neuron_ReLU_name} IS
	-- GENERIC (
 -- 	input_bit:integer:= {str(bits)}; output_bit:integer:= {str(bits)} ;	 n_input:integer:= {str(num_inputs)}
	-- );
	PORT (
	clk, rst, update_weights : IN STD_LOGIC;
	{x}
-- 	{w}
	bias: IN {IO_type}({str(bits -1)} DOWNTO 0) ; -- offset
	------------------------------------------
 	y: OUT {IO_type} ({str(bits - 1)} DOWNTO 0) --output  result
);
-- clk, rst, 
-- inputs, 
-- weigths, 
-- bias, 
-- output
end ENTITY; 

ARCHITECTURE behavior of {neuron_ReLU_name} is
{MAC_component}''' +
# '' + ROM_component + ''
f'''
	SIGNAL out_reg_MAC : {IO_type} (({str(bits - 1)}) DOWNTO 0);	--reg da saida do MAC
'''+
# "	SIGNAL out_reg_MAC : STD_LOGIC_VECTOR(("+ str(bits - 1) + ") DOWNTO 0);	--reg da saida do MAC	\n"
f'''
	{s_reg_x}
 	{s_reg_w}
	SIGNAL reg_bias : {IO_type} ({str(bits - 1)} DOWNTO 0);

 BEGIN
	{PORT_MAP_MAC}
	{NEURON_ReLU_INPUT_NET} 
END behavior;'''
)

# init_text = 1450
# print(top_neuron_ReLU_txt[init_text : (init_text+2000) ]) #Separa em linhas
# print(top_neuron_ReLU_txt[init_text: ]) #Separa em linhas
print(top_neuron_ReLU_txt)

LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;

ENTITY  neuron_comb_barriers_ReLU_3n_8bit_unsigned_mul0_add0_v0_v0 IS
	-- GENERIC (
 -- 	input_bit:integer:= 8; output_bit:integer:= 8 ;	 n_input:integer:= 3
	-- );
	PORT (
	clk, rst, update_weights : IN STD_LOGIC;
	x1, x2, x3: IN signed(7 DOWNTO 0);
-- 	w1, w2, w3: IN signed(7 DOWNTO 0);
	bias: IN signed(7 DOWNTO 0) ; -- offset
	------------------------------------------
 	y: OUT signed (7 DOWNTO 0) --output  result
);
-- clk, rst, 
-- inputs, 
-- weigths, 
-- bias, 
-- output
end ENTITY; 

ARCHITECTURE behavior of neuron_comb_barriers_ReLU_3n_8bit_unsigned_mul0_add0_v0_v0 is

	COMPONENT MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
	PORT (
		clk, rst : IN STD_LOGIC;
		x1, x2, x3: IN signed(7 DOWNTO 0);
 		w1, w2, w3: IN signed(7 DOWNTO 0);
		bias: IN signed(7 DOWNTO 0) ; -- offset 
		------------------------------------------ 
		output: OUT signed(7 D

###### <font color=00FFE6> **Top Neuronio > tree > Texto > Middle NN > Leaky ReLU** </font>

In [139]:
# leaky_shift_text = ('')
# for i in range(0,leaky_attenuation):
#   leaky_shift_text = leaky_shift_text + "sum_result(bits-1) & "
# leaky_shift_text

In [140]:
top_neuron_Leaky_txt = ('''LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;

'''
"ENTITY  "+ neuron_Leaky_name +" IS \n" 
"	-- GENERIC ( \n"
" -- 	input_bit:integer:="+ str(bits) + "; output_bit:integer:="+ str(bits) + ";	 n_input:integer:="+ str(num_inputs) + "\n"
"	-- );"'''
	PORT (
	clk, rst, update_weights : IN STD_LOGIC;'''
'	\n'
'	'+x+'\n'
'-- 	'+w+'\n'
"	bias: IN "+IO_type_str+"(" +str(bits -1)+ " DOWNTO 0) ; -- offset \n"
"	------------------------------------------ \n"
" 	y: OUT " + IO_type_str + "("+ str(bits - 1) + " DOWNTO 0) --output  result" '''
);
-- clk, rst, 
-- inputs, 
-- weigths, 
-- bias, 
-- output
end ENTITY; 

'''
"ARCHITECTURE behavior of "+neuron_Leaky_name+" is		\n"
'' + MAC_component + ''
# '' + ROM_component + ''
"\n"
"	SIGNAL out_reg_MAC : " + IO_type_str + "(("+ str(bits - 1) + ") DOWNTO 0);	--reg da saida do MAC	\n"
# "	SIGNAL out_reg_MAC : STD_LOGIC_VECTOR(("+ str(bits - 1) + ") DOWNTO 0);	--reg da saida do MAC	\n"

'	' + s_reg_x + ''
' 	' + s_reg_w + ' '
"	SIGNAL reg_bias : " + IO_type_str + "("+ str(bits - 1) + " DOWNTO 0); "
'''

 BEGIN \n'''
'	' + PORT_MAP_MAC + '\n' 
# '	' + PORT_MAP_ROM + '\n' 
'	' + NEURON_LeakyReLU_INPUT_NET + '\n' 
'''
END behavior;'''
)

# init_text = 1450
# print(top_neuron_Leaky_txt[init_text : (init_text+2000) ]) #Separa em linhas
# print(top_neuron_Leaky_txt[init_text: ]) #Separa em linhas
print(top_neuron_Leaky_txt)

LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;

ENTITY  neuron_comb_barriers_LeakyReLU_3n_8bit_unsigned_mul0_add0_v0_v0 IS 
	-- GENERIC ( 
 -- 	input_bit:integer:=8; output_bit:integer:=8;	 n_input:integer:=3
	-- );
	PORT (
	clk, rst, update_weights : IN STD_LOGIC;	
	x1, x2, x3: IN signed(7 DOWNTO 0);
-- 	w1, w2, w3: IN signed(7 DOWNTO 0);
	bias: IN unsigned(7 DOWNTO 0) ; -- offset 
	------------------------------------------ 
 	y: OUT unsigned(7 DOWNTO 0) --output  result
);
-- clk, rst, 
-- inputs, 
-- weigths, 
-- bias, 
-- output
end ENTITY; 

ARCHITECTURE behavior of neuron_comb_barriers_LeakyReLU_3n_8bit_unsigned_mul0_add0_v0_v0 is		

	COMPONENT MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
	PORT (
		clk, rst : IN STD_LOGIC;
		x1, x2, x3: IN signed(7 DOWNTO 0);
 		w1, w2, w3: IN signed(7 DOWNTO 0);
		bias: IN signed(7 DOWNTO 0) ; -- offset 
		------------------------------------------ 
		output

#### <font color=yellow> **Top Neuronio > tree > Imprimindo texto** </font>

In [141]:
# init_text = 1450
# print(top_neuron_soft_txt[init_text : (init_text+2000) ]) #Separa em linhas
# print(top_neuron_soft_txt)

In [142]:
#---------------- VHD ----------------#  
if (download_vhd == 1): 
  # ReLU
  with open(f"{path_ReLU}/{neuron_ReLU_name}.vhd", "w") as writer :
    writer.write(top_neuron_ReLU_txt)
    
  # Leaky ReLU
  with open(f"{path_LeakyReLU}/{neuron_Leaky_name}.vhd", "w") as writer :
    writer.write(top_neuron_Leaky_txt)  
    
  # SOFTMAX    
  with open(f"{path_soft}/{neuron_Sigmoid_name}.vhd", "w") as writer :
    writer.write(top_neuron_soft_txt)
    
#---------------- TEXTO ----------------#    
if (download_txt == 1): 
  # ReLU
  with open( f"{path_ReLU}/{neuron_ReLU_name}.txt", "w") as writer :
    writer.write(top_neuron_ReLU_txt)

  # Leaky ReLU
  with open( f"{path_LeakyReLU}/{neuron_Leaky_name}.txt", "w") as writer :
    writer.write(top_neuron_Leaky_txt)
  
  # SOFTMAX    
  with open(f"{path_soft}/{neuron_Sigmoid_name}.txt", "w") as writer :
    writer.write(top_neuron_soft_txt)

### <font color=orange> **Top Neuronio > seq -> Sequencial (multiciclo)** </font>

## <font color=red>**Testbench neuronio**</font>

# <font color=white>Geração de camadas</font>

## <font color=green>**Parâmetros**</font>

In [143]:
# layer_num = 2                             # Número da camada
# neuron_num = layer_dict_hidden['Neurons_number'] # Quantidade de neurônios
# bits = layer_dict_hidden['bits']                 # Quantidade de bits para representação
# num_inputs =  layer_dict_hidden['Inputs_number'] # Quantidade de entradas
# IO_type = layer_dict_hidden['IO_type']           # 1= signed || 0= unsigned

# # ----- MAC -----
# Barriers = layer_dict_hidden['Neuron_arch']['Barriers']  #0 = no Barriers | 1 = with register Barriers (pipeline)
# MAC_type = layer_dict_hidden['Neuron_arch']['MAC_type']            #0 = combinational(tree:árvore) | 1 = Sequential
# Include_MAC_type =  layer_dict_hidden['Neuron_arch']['Include_MAC_type']    #0 = don't inluce | 1 = include 'seq' or 'tree' on vhd_names



# DEBUG = 1              # 1 = mostra outputs no jupyter| 0 = não mostra  
# write_vhd_file = 1                  # =1: cria arquivo .vhd, =0: não cria 

## <font color=green>**Lógicas**</font>

### <font color=orange>Nomes</font>

#### <font color=grey>Camada</font>

In [144]:
#### Nome do arquivo .vhd ####
layer_dict_hidden['Layer_name'] = layer_name(layer_dict_arg = layer_dict_hidden)

if (DEBUG == 1):
  print(layer_dict_hidden['Layer_name'])

camada_ReLU_4neuron_8bits_3n_signed


#### <font color=grey>Neurônio</font>

In [145]:
print(neuron_ReLU_name)
print(neuron_Leaky_name) 
print(neuron_ReLU_name)

neuron_comb_barriers_ReLU_3n_8bit_unsigned_mul0_add0_v0_v0
neuron_comb_barriers_LeakyReLU_3n_8bit_unsigned_mul0_add0_v0_v0
neuron_comb_barriers_ReLU_3n_8bit_unsigned_mul0_add0_v0_v0


In [146]:
# top_neuron_name   = "top_neuron_tree_" + str(bits) + "bits_n" + str(neuron_num) + "_signed"
# top_neuron_name = neuron_ReLU_name
top_neuron_name = layer_dict_hidden['Neuron_arch']['neuron_name']
if (DEBUG == 1):
  print(top_neuron_name)

neuron_comb_ReLU_3n_8bit_signed_mul0_v0_add0_v0


### <font color=orange>Neurônios</font>

In [147]:
a,b = IO_manager([Neuron_IO_dict],bits,num_inputs,onerow = False,tab_space = 2)
print(a)

In [ ]:
layer_dict_hidden['Neuron_arch']['IO']['unique_IO']['IN']['SIGNED_num_inputs']

['w']

In [ ]:
input_sequences_number_choice(
            layer_dict_hidden['Neuron_arch']['IO']['unique_IO']['IN']['SIGNED_num_inputs'], layer_dict_hidden['Neuron_arch']['IO']['unique_IO']['IN']['SIGNED_num_inputs'], False, 1, 'list', 1, 1, False)

('w=> w, \n', ['w'])

In [ ]:
port_map_txt, layer_dict_hidden = layer_neurons_port_map_ALL(layer_dict_arg = layer_dict_hidden,
                               ID_camada = 'c1')
layer_dict_hidden

{'Inputs_number': 3,
 'bits': 8,
 'IO_type': True,
 'Neurons_number': 4,
 'Layer_name': 'camada_ReLU_4neuron_8bits_3n_signed',
 'Layer_number': '',
 'IO': {'IN': {'STD_LOGIC': ['clk', 'rst'],
   'STD_LOGIC_VECTOR': None,
   'SIGNED': ['c1_n0_bias',
    'c1_n1_bias',
    'c1_n2_bias',
    'c1_n3_bias',
    'x1',
    'x2',
    'x3',
    'c1_n0_w1',
    'c1_n0_w2',
    'c1_n0_w3',
    'c1_n1_w1',
    'c1_n1_w2',
    'c1_n1_w3',
    'c1_n2_w1',
    'c1_n2_w2',
    'c1_n2_w3',
    'c1_n3_w1',
    'c1_n3_w2',
    'c1_n3_w3']},
  'OUT': {'STD_LOGIC': None,
   'STD_LOGIC_VECTOR': None,
   'SIGNED': ['c1_n0_y', 'c1_n1_y', 'c1_n2_y', 'c1_n3_y']}},
 'Neuron_arch': {'Inputs_number': <function standard_dicts.<lambda>()>,
  'Bit_WIDTH': <function standard_dicts.<lambda>()>,
  'IO_type': <function standard_dicts.<lambda>()>,
  'neuron_name': 'neuron_comb_ReLU_3n_8bit_signed_mul0_v0_add0_v0',
  'Include_MAC_type': True,
  'IO': {'shared_IO': {'IN': {'STD_LOGIC': ['clk', 'rst'],
     'STD_LOGIC_VECTOR'

## <font color=green>**Escrita do texto**</font>

In [ ]:
# def all_dense_layers_gen(
#     Inputs_number: int,
#     bits: int,
#     IO_type_str: str, # 'signed' or 'unsigned'
#     number_of_layers: int,
#     Layer_Neurons_number_list: list,
#     base_dict_hidden_layers: dict,
#     base_dict_softmax_layer: dict
#     ):
    

#     # alterando parâmetro IO_type_str para True ou False
#     IO_type_str = False
#     if IO_type_str == 'signed':
#         IO_type_str = True

#     # gerando lista para armazenar dicionários de cada camada
#     layers_dict_list = [None]*number_of_layers
    
#     # --------------------------------------------------------
#     """ atribuindo os dicionários base para cada item da lista. 
#     Sendo:
#     dicionário base das 'hidden_layers' = layer_dict_hidden
#     dicionário base da camada softmax = layer_dict_softmax
#         """
#     for i in range(0, len(layers_dict_list)):
#         layers_dict_list[i] = base_dict_hidden_layers
#         if (i == (len(layers_dict_list)-1 )):
#             layers_dict_list[i] = base_dict_softmax_layer
    
#     # --------------------------------------------------------
#     # LOOP PARA ALTERAR ALGUNS PARÂMETROS DOS DICIONÁRIOS DE CADA CAMADA
#     layers_dict_list[0] = layer_dict_gen_base(base_dict = layers_dict_list[0],
#         Inputs_number = Inputs_number,
#         bits = bits,
#         IO_type = IO_type_str, # 1= signed || 0= unsigned
#         Neurons_number = Layer_Neurons_number_list[0],
#         Layer_number = 0
#         )  
    
#     for i in range(1 , len(layers_dict_list)):

#         layers_dict_list[i] = layer_dict_gen_base(base_dict = layers_dict_list[i],
#         Inputs_number = layers_dict_list[i - 1]['Neurons_number'],
#         bits = bits,
#         IO_type = IO_type_str, # 1= signed || 0= unsigned
#         Neurons_number = Layer_Neurons_number_list[i],
#         Layer_number = i
#         )  
        
#     # --------------------------------------------------------
#     # LOOP PARA GERAR TEXTO DE CADA CAMADA CONFORME RESPECTIVO DICIONÁRIO E JÁ SALVAR COMO ARQUIVO '.vhd'
#     for i in range(0 , len(layers_dict_list)):
        
#         # testando gerar texto para primeiro dicionário da 'lista de dicionários'
#         layer_text = layer_gen(layer_dict_arg = layers_dict_list[i], Layer_number = layers_dict_list[i]['Layer_number'])

#         # salvando VHDL
#         if (download_vhd == 1):    
#             with open(f"{path[0]}{layers_dict_list[i]['Layer_name']}.vhd", "w") as writer :
#                 writer.write(layer_text)

# # # imprimindo texto VHDL para análise no Jupyter Notebook
# # if (DEBUG == 1):
# #   print(layer_text)

In [ ]:
# # gerando lista para armazenar dicionários de cada camada
# numero_de_camadas = 4
# layers_dict_list = [None]*numero_de_camadas
# layers_dict_list

In [ ]:
# """ atribuindo os dicionários base para cada item da lista. 
# Sendo:
#  dicionário base das 'hidden_layers' = layer_dict_hidden
#  dicionário base da camada softmax = layer_dict_softmax
#     """
# for i in range(0, len(layers_dict_list)):
#     layers_dict_list[i] = layer_dict_hidden
#     if (i == (len(layers_dict_list)-1 )):
#         layers_dict_list[i] = layer_dict_softmax

# # layers_dict_list[len(layers_dict_list)-1]

In [ ]:
# layers_dict_list[0]

In [ ]:
# Layer_Neurons_number_list = [10,7,5,3]


# # ==== ALTERANDO PARÃMETROS DE DICIONÁRIOS, QUE SÃO CÓPIA DE DICIONÁRIOS BASE (DICIONÁRIOS PADRÃO) ===
# #  ----------------- PRIMEIRA CAMADA -----------------
# # gerando dicionário novo, com base no dicionário layers_dict_list[0]
# # porém layers_dict_list[0] foi copiado do 'dicionário base' ou 'dicionário padrão' que é o 'layer_dict_hidden'
# i = 0
# layers_dict_list[i] = layer_dict_gen_base(base_dict = layers_dict_list[i],
#     Inputs_number = 5,
#     bits = 8,
#     IO_type = True, # 1= signed || 0= unsigned
#     Neurons_number = Layer_Neurons_number_list[i],
#     Layer_number = i
#     )

# layers_dict_list[i]

In [ ]:
# # ----------------- DEMAIS CAMADAS -------------------
# i = 1
# layers_dict_list[i] = layer_dict_gen_base(base_dict = layers_dict_list[i],
#     Inputs_number = layers_dict_list[i - 1]['Neurons_number'],
#     bits = layers_dict_list[0]['bits'],
#     IO_type = layers_dict_list[0]['IO_type'], # 1= signed || 0= unsigned
#     Neurons_number = Layer_Neurons_number_list[i],
#     Layer_number = i
#     )

# layers_dict_list[i]

In [ ]:
# layers_dict_list[i-1]

In [ ]:
# # ----------------- ÚLTIMA CAMADA (SOFTMAX) -----------------
# i = 2
# layers_dict_list[i] = layer_dict_gen_base(base_dict = layers_dict_list[i],
#     Inputs_number = layers_dict_list[i - 1]['Neurons_number'],
#     bits = layers_dict_list[0]['bits'],
#     IO_type = layers_dict_list[0]['IO_type'], # 1= signed || 0= unsigned
#     Neurons_number = Layer_Neurons_number_list[i],
#     Layer_number = i
#     ) 
# layers_dict_list[i]

In [ ]:
# # LOOP PARA GERAR TEXTO DE CADA CAMADA CONFORME RESPECTIVO DICIONÁRIO E JÁ SALVAR COMO ARQUIVO '.vhd'
# for i in range(0 , len(layers_dict_list)):
#     # testando gerar texto para primeiro dicionário da 'lista de dicionários'
#     layer_text = layer_gen(layer_dict_arg = layers_dict_list[i], Layer_number = layers_dict_list[i]['Layer_number'])

#     # salvando VHDL
#     if (download_vhd == 1):    
#         with open(f"{path[0]}{layers_dict_list[i]['Layer_name']}.vhd", "w") as writer :
#             writer.write(layer_text)

# # # imprimindo texto VHDL para análise no Jupyter Notebook
# # if (DEBUG == 1):
# #   print(layer_text)

In [ ]:
# layers_dict_list[0]

In [ ]:
# # testando gerar texto para primeiro dicionário da 'lista de dicionários'
# layer_text = layer_gen(layer_dict_arg = layers_dict_list[0], Layer_number = 0)

# # salvando VHDL
# if (download_vhd == 1):    
#   with open(f"{path[0]}{layers_dict_list[0]['Layer_name']}.vhd", "w") as writer :
#     writer.write(layer_text)

# # imprimindo texto VHDL para análise no Jupyter Notebook
# if (DEBUG == 1):
#   print(layer_text)

## <font color=green>**Criando .vhd**</font>

In [ ]:
# import Create_Folders

In [ ]:
# path = Create_Folders.create_folder(['NNs/Camadas/'], 1, download_vhd) #(path_list, path_level, create)
# if (DEBUG == 1 ):
#   print(f"Relu: {path[0]}")

In [ ]:
# if (download_vhd == 1):    
#   with open(f"{path[0]}{layer_dict_hidden['Layer_name']}.vhd", "w") as writer :
#     writer.write(layer_text)

In [ ]:
layers_dict_list = []

layers_dict_list = all_dense_layers_gen(
    Inputs_number = 5,
    bits = 8,
    IO_type_str = 'signed', # 'signed' or 'unsigned'
    number_of_layers = 4,
    Layer_Neurons_number_list = [3,3,2,2],
    base_dict_hidden_layers = layer_dict_hidden,
    base_dict_softmax_layer = layer_dict_softmax,
    path = 'NNs/Camadas/',
    download_vhd = download_vhd,
    gen_dead_neurons = True # gera neurônios mortos
)

tentativa dic 1
Error entity(): Formato de dicionário inválido para num_inputs
tentativa dic 1
Error entity(): Formato de dicionário inválido para num_inputs
tentativa dic 1
Error entity(): Formato de dicionário inválido para num_inputs
tentativa dic 1
All Dense Layers Created!!


In [ ]:
# layers_dict_list[0]

In [ ]:
lista_n = [10,7,5,3]

In [ ]:
np.max(lista_n)

10

In [ ]:
i = 0
# layers_dict_list[i]
layers_dict_list[i]['IO']['OUT']

{'STD_LOGIC': None,
 'STD_LOGIC_VECTOR': None,
 'SIGNED': ['c0_n0_y', 'c0_n1_y', 'c0_n2_y']}

In [ ]:
i = 1
# layers_dict_list[i]
layers_dict_list[i]['IO']['IN']

{'STD_LOGIC': ['clk', 'rst'],
 'STD_LOGIC_VECTOR': None,
 'SIGNED': ['c1_n0_bias',
  'c1_n1_bias',
  'c1_n2_bias',
  'x1',
  'x2',
  'x3',
  'c1_n0_w1',
  'c1_n0_w2',
  'c1_n0_w3',
  'c1_n1_w1',
  'c1_n1_w2',
  'c1_n1_w3',
  'c1_n2_w1',
  'c1_n2_w2',
  'c1_n2_w3']}

In [ ]:
layers_dict_list[i]['Layer_name']

'camada1_ReLU_3neuron_8bits_3n_signed'

In [ ]:
layers_dict_list[2]['IO']

{'IN': {'STD_LOGIC': ['clk', 'rst'],
  'STD_LOGIC_VECTOR': None,
  'SIGNED': ['c2_n0_bias',
   'c2_n1_bias',
   'c2_n2_bias',
   'x1',
   'x2',
   'x3',
   'c2_n0_w1',
   'c2_n0_w2',
   'c2_n0_w3',
   'c2_n1_w1',
   'c2_n1_w2',
   'c2_n1_w3',
   'c2_n2_w1',
   'c2_n2_w2',
   'c2_n2_w3']},
 'OUT': {'STD_LOGIC': None,
  'STD_LOGIC_VECTOR': None,
  'SIGNED': ['c2_n0_y', 'c2_n1_y', 'c2_n2_y']}}

In [ ]:
i = 0
layers_dict_list[i]['IO']['OUT']

{'STD_LOGIC': None,
 'STD_LOGIC_VECTOR': None,
 'SIGNED': ['c0_n0_y', 'c0_n1_y', 'c0_n2_y']}

In [ ]:
i = 1
layers_dict_list[i]['IO']['OUT']

{'STD_LOGIC': None,
 'STD_LOGIC_VECTOR': None,
 'SIGNED': ['c1_n0_y', 'c1_n1_y', 'c1_n2_y']}

In [ ]:
layers_dict_list[i]['Layer_name']

'camada1_ReLU_3neuron_8bits_3n_signed'

In [ ]:
i = 2
layers_dict_list[i]['IO']['OUT']

{'STD_LOGIC': None,
 'STD_LOGIC_VECTOR': None,
 'SIGNED': ['c2_n0_y', 'c2_n1_y', 'c2_n2_y']}

In [ ]:
layers_dict_list[i]['Layer_name']

'camada2_ReLU_3neuron_8bits_3n_signed'

# outros

In [ ]:
# EXEMPLOS:
# print(input_sequences('x',num_inputs,'string',1,1)) # (sequence_inputs,'string',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences(['x','w'] ,num_inputs,'list',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences('x' ,num_inputs,'string',1,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences('x' _id ,num_inputs, list_or_string, port_map, new_line)

# print(input_sequences('sx' ,num_inputs,'string',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences('sw' ,num_inputs,'string',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences('val_x' ,num_inputs,'string',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
# print(input_sequences('val_w' ,num_inputs,'string',0,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line)
print(input_sequences(['x','w'] ,num_inputs,'list',1,0)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line))
print(input_sequences(['x','w'] ,num_inputs,'list',1,1)) # (sequence_id ,num_inputs, list_or_string, port_map, new_line))

['x1=>x1', 'x2=>x2', 'x3=>x3', 'w1=>w1', 'w2=>w2', 'w3=>w3']
x1=>x1, 
x2=>x2, 
x3=>x3, 
w1=>w1, 
w2=>w2, 
w3=>w3, 



# vendo POO

##### <font color=cyan> **Top Neuronio > tree > Função texto** </font>

In [ ]:
layers_dict_list[0]['Neuron_arch']['Bit_WIDTH']()

8

In [ ]:
print(entity(name = layers_dict_list[0]['Neuron_arch']['neuron_name'], 
             bits = layers_dict_list[0]['Neuron_arch']['Bit_WIDTH'](), 
             num_inputs = layers_dict_list[0]['Neuron_arch']['Inputs_number'](),
             IO_dict_list = [layers_dict_list[0]['Neuron_arch']['IO']['shared_IO'], layers_dict_list[0]['Neuron_arch']['IO']['unique_IO']]
             ))

tentativa dic 1
ENTITY  neuron_comb_ReLU_5n_8bit_signed_mul0_v0_add0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    y: OUT signed(7 DOWNTO 0)
    );
end ENTITY;


In [ ]:
print(MAC_component)


	COMPONENT MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
	PORT (
		clk, rst : IN STD_LOGIC;
		x1, x2, x3: IN signed(7 DOWNTO 0);
 		w1, w2, w3: IN signed(7 DOWNTO 0);
		bias: IN signed(7 DOWNTO 0) ; -- offset 
		------------------------------------------ 
		output: OUT signed(7 DOWNTO 0)
	); -- clk, rst, inputs, weigths, bias, output
	END COMPONENT;                            



In [ ]:
neuron_ReLu_POO_txt = (f'''LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;

{entity(name = layers_dict_list[0]['Neuron_arch']['neuron_name'], 
             bits = layers_dict_list[0]['Neuron_arch']['Bit_WIDTH'](), 
             num_inputs = layers_dict_list[0]['Neuron_arch']['Inputs_number'](),
             IO_dict_list = [layers_dict_list[0]['Neuron_arch']['IO']['shared_IO'], layers_dict_list[0]['Neuron_arch']['IO']['unique_IO']]
             )}


ARCHITECTURE behavior of {neuron_ReLU_name} is
{MAC_component}''' +
# '' + ROM_component + ''
f'''
	SIGNAL out_reg_MAC : {IO_type} (({str(bits - 1)}) DOWNTO 0);	--reg da saida do MAC
'''+
# "	SIGNAL out_reg_MAC : STD_LOGIC_VECTOR(("+ str(bits - 1) + ") DOWNTO 0);	--reg da saida do MAC	\n"
f'''
	{s_reg_x}
 	{s_reg_w}
	SIGNAL reg_bias : {IO_type} ({str(bits - 1)} DOWNTO 0);

 BEGIN
	{PORT_MAP_MAC}
	{NEURON_ReLU_INPUT_NET} 
END behavior;'''
)

# init_text = 1450
# print(top_neuron_soft_txt[init_text : (init_text+2000) ]) #Separa em linhas
# print(top_neuron_soft_txt[init_text: ]) #Separa em linhas
print(neuron_ReLu_POO_txt)

tentativa dic 1
LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;

ENTITY  neuron_comb_ReLU_5n_8bit_signed_mul0_v0_add0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    y: OUT signed(7 DOWNTO 0)
    );
end ENTITY;


ARCHITECTURE behavior of neuron_comb_barriers_ReLU_3n_8bit_unsigned_mul0_add0_v0_v0 is

	COMPONENT MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
	PORT (
		clk, rst : IN STD_LOGIC;
		x1, x2, x3: IN signed(7 DOWNTO 0);
 		w1, w2, w3: IN signed(7 DOWNTO 0);
		bias: IN signed(7 DOWNTO 0) ; -- offset 
		------------------------------------------ 
		output: OUT signed(7 DOWNTO 0)
	); -- clk, rst, inputs, weigths, bias, output
	END COMPONENT;                            

	SIGNAL out_reg_MAC : signed ((7) DOWNTO 0);	--reg da saida do MAC

	SI

In [ ]:
same_level_dir = os.path.abspath(".")   # SAME LEVEL

#---------------- VHD ----------------#  
if (download_vhd == 1): 
  # ReLU
  with open(f"{same_level_dir}/POO_Neuron.vhd", "w") as writer :
    writer.write(neuron_ReLu_POO_txt)
    
    

In [ ]:
print(MAC_component)


	COMPONENT MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
	PORT (
		clk, rst : IN STD_LOGIC;
		x1, x2, x3: IN signed(7 DOWNTO 0);
 		w1, w2, w3: IN signed(7 DOWNTO 0);
		bias: IN signed(7 DOWNTO 0) ; -- offset 
		------------------------------------------ 
		output: OUT signed(7 DOWNTO 0)
	); -- clk, rst, inputs, weigths, bias, output
	END COMPONENT;                            



In [ ]:
layers_dict_list[0]['Neuron_arch']

{'Inputs_number': <function standard_dicts.<lambda>()>,
 'Bit_WIDTH': <function standard_dicts.<lambda>()>,
 'IO_type': <function standard_dicts.<lambda>()>,
 'neuron_name': 'neuron_comb_ReLU_5n_8bit_signed_mul0_v0_add0_v0',
 'Include_MAC_type': True,
 'IO': {'shared_IO': {'IN': {'STD_LOGIC': ['clk', 'rst'],
    'STD_LOGIC_VECTOR': None,
    'SIGNED': None,
    'STD_LOGIC_num_inputs': None,
    'STD_LOGIC_VECTOR_num_inputs': None,
    'SIGNED_num_inputs': ['x']},
   'OUT': {'STD_LOGIC': None,
    'STD_LOGIC_VECTOR': None,
    'SIGNED': None,
    'STD_LOGIC_VECTOR_num_inputs': None,
    'STD_LOGIC_num_inputs': None,
    'SIGNED_num_inputs': None}},
  'unique_IO': {'IN': {'STD_LOGIC': None,
    'STD_LOGIC_VECTOR': None,
    'SIGNED': ['bias'],
    'STD_LOGIC_num_inputs': None,
    'STD_LOGIC_VECTOR_num_inputs': None,
    'SIGNED_num_inputs': ['w']},
   'OUT': {'STD_LOGIC': None,
    'STD_LOGIC_VECTOR': None,
    'SIGNED': ['y'],
    'STD_LOGIC_VECTOR_num_inputs': None,
    'STD_LOGIC_num

In [ ]:
entity(name = layers_dict_list[0]['Neuron_arch']['neuron_name'], 
             bits = layers_dict_list[0]['Neuron_arch']['Bit_WIDTH'](), 
             num_inputs = layers_dict_list[0]['Neuron_arch']['Inputs_number'](),
             IO_dict_list = [layers_dict_list[0]['Neuron_arch']['IO']['shared_IO'], layers_dict_list[0]['Neuron_arch']['IO']['unique_IO']])

tentativa dic 1


'ENTITY  neuron_comb_ReLU_5n_8bit_signed_mul0_v0_add0_v0 IS\n  PORT (\n    clk, rst: IN STD_LOGIC;\n    bias: IN signed(7 DOWNTO 0);\n    x1, x2, x3: IN signed(7 DOWNTO 0);\n    w1, w2, w3: IN signed(7 DOWNTO 0);\n    ----------------------------------------------\n    y: OUT signed(7 DOWNTO 0)\n    );\nend ENTITY;'

In [ ]:
print(entity(name = vhd_name('MAC', bits, IO_type, num_inputs, Barriers, MAC_type, Include_MAC_type),
           bits= layers_dict_list[0]['Neuron_arch']['Bit_WIDTH'](),
           num_inputs= layers_dict_list[0]['Neuron_arch']['Inputs_number'](),
           IO_dict_list =  [layers_dict_list[0]['Neuron_arch']['IO']['shared_IO'], layers_dict_list[0]['Neuron_arch']['IO']['unique_IO']]))

tentativa dic 1
ENTITY  MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    y: OUT signed(7 DOWNTO 0)
    );
end ENTITY;


In [ ]:
layers_dict_list[0]['Neuron_arch']

{'Inputs_number': <function standard_dicts.<lambda>()>,
 'Bit_WIDTH': <function standard_dicts.<lambda>()>,
 'IO_type': <function standard_dicts.<lambda>()>,
 'neuron_name': 'neuron_comb_ReLU_5n_8bit_signed_mul0_v0_add0_v0',
 'Include_MAC_type': True,
 'IO': {'shared_IO': {'IN': {'STD_LOGIC': ['clk', 'rst'],
    'STD_LOGIC_VECTOR': None,
    'SIGNED': None,
    'STD_LOGIC_num_inputs': None,
    'STD_LOGIC_VECTOR_num_inputs': None,
    'SIGNED_num_inputs': ['x']},
   'OUT': {'STD_LOGIC': None,
    'STD_LOGIC_VECTOR': None,
    'SIGNED': None,
    'STD_LOGIC_VECTOR_num_inputs': None,
    'STD_LOGIC_num_inputs': None,
    'SIGNED_num_inputs': None}},
  'unique_IO': {'IN': {'STD_LOGIC': None,
    'STD_LOGIC_VECTOR': None,
    'SIGNED': ['bias'],
    'STD_LOGIC_num_inputs': None,
    'STD_LOGIC_VECTOR_num_inputs': None,
    'SIGNED_num_inputs': ['w']},
   'OUT': {'STD_LOGIC': None,
    'STD_LOGIC_VECTOR': None,
    'SIGNED': ['y'],
    'STD_LOGIC_VECTOR_num_inputs': None,
    'STD_LOGIC_num

# Função MAC

## Parametros iniciais

In [ ]:
IO_type
bits
num_inputs

n_bin
MAC_type
Include_MAC_type
# -----------------
x,w,sx,sw,sx_tb,sw_tb,val_x,val_w,val_x_tb,val_w_tb = all_inputs_signals(['x','w'],num_inputs,bits,IO_type,0)
# --------------------
spacebar_1  = 1
rst_space   = 3 #rst_receive (quando rst = '1')
clk_space   = 4 #quando clk = '1'
# ------------------
# if (IO_type == 1):      #versão com sinal
#   IO_type = 'signed'
#   std_logic_conversion = IO_type
# else:                     #versão sem sinal
#   IO_type = 'STD_LOGIC_VECTOR'
#   std_logic_conversion = 'unsigned'
# -------------------------------------
MAC_IO_dict = {
  'IN': {
  # INPUTS
  'STD_LOGIC': ['clk', 'rst'] , #
  'STD_LOGIC_VECTOR': None, #['A','B'],
  # 'STD_LOGIC_VECTOR': ['A','B'],
  'STD_LOGIC_num_inputs': None, #['C','D'],
  'STD_LOGIC_VECTOR_num_inputs': None,
  'SIGNED': ['bias'],
  'SIGNED_num_inputs': ['x','w']
  },
  
  'OUT' : {
  #  OUTPUTS
  'STD_LOGIC': None , #
  'STD_LOGIC_VECTOR': None, #['A','B'],
  # 'STD_LOGIC_num_inputs': None, #['C','D'],
  'STD_LOGIC_num_inputs': None, #['C','D'],
  'STD_LOGIC_VECTOR_num_inputs': None,
  'SIGNED': ['output'],
  'SIGNED_num_inputs': None
    
  }
}
# ------------------------------------- || ---------------------

## entity + arch

In [ ]:
MAC_name = 'MAC'

layer_dict  = layers_dict_list[0]

vhd_txt = (f'''LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.numeric_std.ALL;

{entity(name = MAC_name, 
             bits = layer_dict['Neuron_arch']['Bit_WIDTH'], 
             num_inputs = layer_dict['Neuron_arch']['Inputs_number'],
             IO_dict_list = [layer_dict['Neuron_arch']['IO']['shared_IO'], layer_dict['Neuron_arch']['IO']['unique_IO']]
             )}

ARCHITECTURE arch OF  {MAC_name}  IS 
BEGIN
{MAC_Tree_Txt_Gen_Barriers(ReceiveStr_list = [signal_receive_string_MAC, signal_receive_string_MAC, clk_receive_string_MAC])}
END ARCHITECTURE;
'''
)
print(vhd_txt)


tentativa dic 1
LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.numeric_std.ALL;

ENTITY  MAC IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    y: OUT signed(7 DOWNTO 0)
    );
end ENTITY;

ARCHITECTURE arch OF  MAC  IS 
BEGIN

  sx1 <= x1;
  sx2 <= x2;
  sx3 <= x3;
  sw1 <= w1;
  sw2 <= w2;
  sw3 <= w3;
  sbias <= bias;
	------------------
	PROCESS (rst, clk) 
	BEGIN
		IF (rst = '1') THEN 
  sx1 <= x1;
  sx2 <= x2;
  sx3 <= x3;
  sw1 <= w1;
  sw2 <= w2;
  sw3 <= w3;
		ELSE
			IF (clk'event AND clk = '1') THEN --se tem evento de clock 
        output <= signed(soma_3(15 DOWNTO 8));
        mult_1 <= (signed(sx1) * signed(sw1));
        mult_2 <= (signed(sx2) * signed(sw2));
        mult_3 <= (signed(sx3) * signed(sw3));
        soma_1 <=  mult_1 + mult_2;
        soma_2 <=  soma_1 + mult_3;
        soma_3 <= soma_2 + sbias; 


	

## MAC_logic()

In [ ]:
def MAC_logic(bits: int,
           num_inputs: int #,
          #  IO_dict_list: list
           )-> str:
    
    IO_type = 'signed'

    x,w,sx,sw,sx_tb,sw_tb,val_x,val_w,val_x_tb,val_w_tb = all_inputs_signals(['x','w'],num_inputs,bits,IO_type,is_list = 0)
    
    spacebar_1  = 1
    rst_space   = 3 #rst_receive (quando rst = '1')
    clk_space   = 4 #quando clk = '1

    output_sum = seq_input_output(['output'],'OUT',IO_type,bits,'string','end')

    signal_receive_string = signal_receive_input(num_inputs,1)
# print(signal_receive_string)
    clk_receive_string, clk_receive = clk_receive_logic(
  clk_space = 4,
  IO_type = 'signed',
  bits = 8,
  n_bin = 8,
  sum_sequence= ['soma_1', 'soma_2', 'soma_3'],
  mult_sequence = ['mult_1', 'mult_2', 'mult_3'])

# print(clk_receive_string)
# print(clk_receive)   

    s_sum,s_mult,signal_receive_string_MAC,rst_receive_string_MAC,clk_receive_string_MAC = arvore_de_soma_strings(num_inputs,bits,n_bin,IO_type,rst_space,clk_space)

    if (DEBUG == 1):
      print(rst_receive_string_MAC)
      print('------------------------')
      print(clk_receive_string_MAC)


# ========================================================================

    IO, traço = IO_manager([MAC_IO_dict],bits,num_inputs,1,2)

    if (DEBUG == 1):
      print(IO)
  # print(traço)

    MAC_signals = (f'''
        SIGNAL sbias : {IO_type}({str(bits -1)} DOWNTO 0);
	SIGNAL sum_all : signed( {str((2*bits) -1)} DOWNTO 0);
	{sx}
	{sw}
	{s_sum}
	{s_mult}''')
    print(MAC_signals)


# ----------------- SEM BARREIRAS --------------

#SOMATÓRIO -> operação de soma ponderada no sum.vhd
    sum_all = []
    i= 0

    while True:
      if (i == 0):
        sum_all.append("sum_all <= (("+std_logic_conversion+"(sx"+str(i+1)+")*"+std_logic_conversion+"(sw"+str(i+1)+"))+")
        i= i+1
  
      else:
        if (i < num_inputs-1):
          sum_all.append("("+std_logic_conversion+"(sx"+str(i+1)+")*"+std_logic_conversion+"(sw"+str(i+1)+"))+")
          i= i+1
        else:
          sum_all.append("("+std_logic_conversion+"(sx"+str(i+1)+")*"+std_logic_conversion+"(sw"+str(i+1)+"))+"+std_logic_conversion+"(sbias));")
          i= 0
          break

    sum_all_string = '\n'.join(map(str, sum_all)) 

    if (DEBUG == 1):
      print(sum_all_string)

    space = "  "
    signal_receive = []

    for i in signal_receive_string_MAC.splitlines():
      signal_receive.append(space + i)
  # print(space + i)
  
    signal_receive = '\n'.join(map(str, signal_receive)) 
    signal_receive_string = signal_receive

    if (DEBUG == 1):
      print(signal_receive_string)

  # return TXT_MAC

MAC_logic(bits =2, num_inputs=3,)

      output <= (OTHERS => '0');
      soma_1 <= (OTHERS => '0');
      soma_2 <= (OTHERS => '0');
      soma_3 <= (OTHERS => '0');
      mult_1 <= (OTHERS => '0');
      mult_2 <= (OTHERS => '0');
      mult_3 <= (OTHERS => '0');
------------------------
        output <= signed(soma_3(9 DOWNTO 8));
        mult_1 <= (signed(sx1) * signed(sw1));
        mult_2 <= (signed(sx2) * signed(sw2));
        mult_3 <= (signed(sx3) * signed(sw3));
        soma_1 <=  mult_1 + mult_2;
        soma_2 <=  soma_1 + mult_3;
        soma_3 <= soma_2 + sbias; 


    clk, rst: IN STD_LOGIC;
    bias: IN signed(1 DOWNTO 0);
    x1, x2, x3: IN signed(1 DOWNTO 0);
    w1, w2, w3: IN signed(1 DOWNTO 0);
    ----------------------------------------------
    output: OUT signed(1 DOWNTO 0)

        SIGNAL sbias : signed(1 DOWNTO 0);
	SIGNAL sum_all : signed( 3 DOWNTO 0);
	SIGNAL sx1, sx2, sx3: signed(1 DOWNTO 0); 
	SIGNAL sw1, sw2, sw3: signed(1 DOWNTO 0); 
	SIGNAL soma_1, soma_2, soma_3: signed(3 DOWNTO 0); 

## MAC_IO_dict

In [ ]:
# -------------------
MAC_IO_dict = {
  
  'IN': {
  # INPUTS
  'STD_LOGIC': ['clk', 'rst'] , #
  'STD_LOGIC_VECTOR': None, #['A','B'],
  # 'STD_LOGIC_VECTOR': ['A','B'],
  'STD_LOGIC_num_inputs': None, #['C','D'],
  'STD_LOGIC_VECTOR_num_inputs': None,
  'SIGNED': ['bias'],
  'SIGNED_num_inputs': ['x','w']
  },
  
  'OUT' : {
  #  OUTPUTS
  'STD_LOGIC': None , #
  'STD_LOGIC_VECTOR': None, #['A','B'],
  # 'STD_LOGIC_num_inputs': None, #['C','D'],
  'STD_LOGIC_num_inputs': None, #['C','D'],
  'STD_LOGIC_VECTOR_num_inputs': None,
  'SIGNED': ['output'],
  'SIGNED_num_inputs': None
    
    }
}
if (DEBUG == 1):
  print(MAC_IO_dict)

{'IN': {'STD_LOGIC': ['clk', 'rst'], 'STD_LOGIC_VECTOR': None, 'STD_LOGIC_num_inputs': None, 'STD_LOGIC_VECTOR_num_inputs': None, 'SIGNED': ['bias'], 'SIGNED_num_inputs': ['x', 'w']}, 'OUT': {'STD_LOGIC': None, 'STD_LOGIC_VECTOR': None, 'STD_LOGIC_num_inputs': None, 'STD_LOGIC_VECTOR_num_inputs': None, 'SIGNED': ['output'], 'SIGNED_num_inputs': None}}


## txt SEM BARREIRA

In [ ]:
# num_inputs

In [ ]:
[MAC_IO_dict]

[{'IN': {'STD_LOGIC': ['clk', 'rst'],
   'STD_LOGIC_VECTOR': None,
   'STD_LOGIC_num_inputs': None,
   'STD_LOGIC_VECTOR_num_inputs': None,
   'SIGNED': ['bias'],
   'SIGNED_num_inputs': ['x', 'w']},
  'OUT': {'STD_LOGIC': None,
   'STD_LOGIC_VECTOR': None,
   'STD_LOGIC_num_inputs': None,
   'STD_LOGIC_VECTOR_num_inputs': None,
   'SIGNED': ['output'],
   'SIGNED_num_inputs': None}}]

In [ ]:
IO_type_str

'unsigned'

In [ ]:
# sx

In [ ]:
# sw

In [ ]:
signal_receive_string = signal_receive_input(num_inputs,1)
print(signal_receive_string)

  sx1 <= x1;
  sx2 <= x2;
  sx3 <= x3;
  sw1 <= w1;
  sw2 <= w2;
  sw3 <= w3;


In [ ]:
sum_all_string = MAC_Txt_Gen(IO_type='signed', num_inputs=3,
      receive_sum_all='sum_all', DEBUG=False)
print(sum_all_string)

sum_all <= ((signed(sx1)*signed(sw1))+
(signed(sx2)*signed(sw2))+
(signed(sx3)*signed(sw3))+ signed(sbias));


In [ ]:


# ========================================================================
# if (Barriers == 0): # SEM BARREIRA
# TXT_MAC = ''
TXT_MAC = (f'''
LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;
--use work.STD_DT.ALL;

{entity(sum_name, bits, num_inputs,[MAC_IO_dict])}

ARCHITECTURE behavior of {sum_name} is
	SIGNAL sbias : ''' + IO_type_str + '''(''' +str(bits -1)+ ''' DOWNTO 0);
	SIGNAL sum_all : signed('''+ str((2*bits) -1) + ''' DOWNTO 0);
	'''+sx+'''
	'''+sw+'''

BEGIN
''' ""+signal_receive_string+"" '''
	sbias <= bias;
 
	'''+sum_all_string+'''

	PROCESS (rst, clk)
	BEGIN
		IF (rst = '1') THEN
			output <= (OTHERS => '0');
		ELSE
			IF (clk'event AND clk = '1') THEN --se tem evento de clock 
				output <= sum_all('''+ str((2*n_bin) -1) + ''' DOWNTO ('''+ str(n_bin) + '''));

			END IF;
		END IF;
	END PROCESS;

END behavior;

''')
if (DEBUG == 1):
	print(TXT_MAC)



tentativa dic 1

LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.std_logic_unsigned.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;
--use work.STD_DT.ALL;

ENTITY  MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    output: OUT signed(7 DOWNTO 0)
    );
end ENTITY;

ARCHITECTURE behavior of MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 is
	SIGNAL sbias : unsigned(7 DOWNTO 0);
	SIGNAL sum_all : signed(15 DOWNTO 0);
	SIGNAL sx1, sx2, sx3: signed(7 DOWNTO 0); 
	SIGNAL sw1, sw2, sw3: signed(7 DOWNTO 0); 

BEGIN
  sx1 <= x1;
  sx2 <= x2;
  sx3 <= x3;
  sw1 <= w1;
  sw2 <= w2;
  sw3 <= w3;
	sbias <= bias;
 
	sum_all <= ((signed(sx1)*signed(sw1))+
(signed(sx2)*signed(sw2))+
(signed(sx3)*signed(sw3))+ signed(sbias));

	PROCESS (rst, clk)
	BEGIN
		IF (rst = '1') THEN
			output 

## txt COM BARREIRA

In [ ]:
# # ----------------- COM BARREIRAS --------------

# # TXT_MAC = ''
# # if (Barriers == 1): # COM BARREIRA
# def MAC_Tree_Txt_Gen_Barriers(ReceiveStr_list: list = [signal_receive_string_MAC, rst_receive_string_MAC, clk_receive_string_MAC]):
#     """Função para gerar o texto
#     Exemplo: 
#     print(MAC_Tree_Txt_Gen_Barriers()):

#         sx1 <= x1;
#         sx2 <= x2;
#         sx3 <= x3;
#         sw1 <= w1;
#         sw2 <= w2;
#         sw3 <= w3;
#         sbias <= bias;
#         ------------------
#         PROCESS (rst, clk) 
#         BEGIN
#                 IF (rst = '1') THEN 
#                     sx1 <= x1;
#                     sx2 <= x2;
#                     sx3 <= x3;
#                     sw1 <= w1;
#                     sw2 <= w2;
#                     sw3 <= w3;
#                 ELSE
#                         IF (clk'event AND clk = '1') THEN --se tem evento de clock 
#                             output <= signed(soma_3(15 DOWNTO 8));
#                             mult_1 <= (signed(sx1) * signed(sw1));
#                             mult_2 <= (signed(sx2) * signed(sw2));
#                             mult_3 <= (signed(sx3) * signed(sw3));
#                             soma_1 <=  mult_1 + mult_2;
#                             soma_2 <=  soma_1 + mult_3;
#                             soma_3 <= soma_2 + sbias; 
#                         END IF;
#                 END IF;
#         END PROCESS;

#     Args:
#         ReceiveStr_list (list, optional): Textos de sinal ou multiplicação gerados através da função 'arvore_de_soma_strings()':
#             s_sum,s_mult,signal_receive_string_MAC,rst_receive_string_MAC,clk_receive_string_MAC = arvore_de_soma_strings(num_inputs,bits,n_bin,IO_type,rst_space,clk_space)
#             Defaults to [signal_receive_string_MAC, signal_receive_string_MAC, clk_receive_string_MAC].
#     """

#     MAC_Tree_Txt = (f'''
# {ReceiveStr_list[0]}
# sbias <= bias;
# ------------------
# PROCESS (rst, clk) 
# BEGIN
#     IF (rst = '1') THEN 
# {ReceiveStr_list[1]}
#     ELSE
#         IF (clk'event AND clk = '1') THEN --se tem evento de clock 
# {ReceiveStr_list[2]}
#         END IF;
#     END IF;
# END PROCESS;
# END behavior;'''
#                     )
#     return MAC_Tree_Txt


# MAC_Tree_Txt_Gen_Barriers()


## Testando

In [ ]:
# spacebar_1  = 1
# rst_space   = 3 #rst_receive (quando rst = '1')
# clk_space   = 4 #quando clk = '1

# s_sum,s_mult,signal_receive_string_MAC,rst_receive_string_MAC,clk_receive_string_MAC = arvore_de_soma_strings(num_inputs,bits,n_bin,IO_type,rst_space,clk_space)

In [ ]:
# x,w,sx,sw,sx_tb,sw_tb,val_x,val_w,val_x_tb,val_w_tb = all_inputs_signals(inputs_list = ['x','w'],
# num_inputs = num_inputs,bits = bits,IO_type = IO_type, is_list = 0)


# MAC_signals = (f'''
#         SIGNAL sbias : {IO_type}({str(bits -1)} DOWNTO 0);
# 	SIGNAL sum_all : signed( {str((2*bits) -1)} DOWNTO 0);
# 	{sx}
# 	{sw}
# 	{s_sum}
# 	{s_mult}''')
# print(MAC_signals)

In [ ]:
# MAC_name = 'MAC'

# layer_dict  = layers_dict_list[0]

# vhd_txt = (f'''LIBRARY ieee;
# USE ieee.std_logic_1164.ALL;
# USE ieee.numeric_std.ALL;
# USE ieee.math_real.ALL;

# {entity(name = MAC_name, 
#              bits = layer_dict['Neuron_arch']['Bit_WIDTH'], 
#              num_inputs = layer_dict['Neuron_arch']['Inputs_number'],
#              IO_dict_list = [layer_dict['Neuron_arch']['IO']['shared_IO'], layer_dict['Neuron_arch']['IO']['unique_IO']]
#              )}
             
# ARCHITECTURE arch OF  {MAC_name}  IS
# {MAC_signals}
# BEGIN
# {MAC_Tree_Txt_Gen_Barriers(ReceiveStr_list = [signal_receive_string_MAC, rst_receive_string_MAC, clk_receive_string_MAC])}
# END ARCHITECTURE;
# '''
# )
# print(vhd_txt)


In [ ]:
# spacebar_1  = 1
# rst_space   = 3 #rst_receive (quando rst = '1')
# clk_space   = 4 #quando clk = '1

# def MAC_Tree_Barriers_TxtGen(MAC_name = 'MAC',
#                             name_with_parameters: bool = False,
#                             rst_space:int = 3, 
#                             clk_space:int = 4,
#                             layer_dict:dict  = layers_dict_list[0]) -> str:
#     """Função para gerar o texto de todo o VHDL do MAC versão árvore, para uma versão com barreiras temporais de registradores entre cada camada da árvore de multiplicações e somas.

#     Args:
#         MAC_name (str, optional): Nome do arquivo VHDL. Defaults to 'MAC'.
#         name_with_parameters (bool, optional): Se você deseja que o nome contenha algumas descrições de parâmetros. Defaults to False.
#         rst_space (int, optional): Espaço de indexação (apenas para organização visual), na parte de declaração quando acontece o 'reset'. Defaults to 3.
#         clk_space (int, optional): Espaço de indexação (apenas para organização visual) na parte de declaração de cada ciclo de clock. Defaults to 4.
#         layer_dict (dict, optional): dicionário da camada. Defaults to layers_dict_list[0].
#     """
    
#     # Adiciona parâmetros no nome, caso (name_with_parameters == True)
#     if(name_with_parameters):
#         MAC_name = vhd_name(MAC_name, bits, IO_type, num_inputs, Barriers, MAC_type, Include_MAC_type)

#     # pegando dados do dicionário
#     bits = layer_dict['Neuron_arch']['Bit_WIDTH']()
#     n_bin = layer_dict['Neuron_arch']['Activation_function']['Sigmoid']['Memory']['input_mem_bits']()
#     num_inputs = layer_dict['Neuron_arch']['Inputs_number']()
    
#     if layer_dict['Neuron_arch']['IO_type']():
#         IO_type = 'signed'
#     else: IO_type = 'unsigned'


#     s_sum,s_mult,signal_receive_string_MAC,rst_receive_string_MAC,clk_receive_string_MAC = arvore_de_soma_strings(num_inputs,bits,n_bin,IO_type,rst_space,clk_space)

#     x,w,sx,sw,sx_tb,sw_tb,val_x,val_w,val_x_tb,val_w_tb = all_inputs_signals(inputs_list = ['x','w'],
# num_inputs = num_inputs,bits = bits,IO_type = IO_type, is_list = 0)

#     #  ---------- SINAIS ----------
#     MAC_signals = (f'''
#         SIGNAL sbias : {IO_type}({str(bits -1)} DOWNTO 0);
# 	SIGNAL sum_all : signed( {str((2*bits) -1)} DOWNTO 0);
# 	{sx}
# 	{sw}
# 	{s_sum}
# 	{s_mult}''')

#     # ---------- TEXTO VHDL ----------
#     vhd_txt = (f'''LIBRARY ieee;
# USE ieee.std_logic_1164.ALL;
# USE ieee.numeric_std.ALL;
# USE ieee.math_real.ALL;
# {entity(name = MAC_name, 
#              bits = layer_dict['Neuron_arch']['Bit_WIDTH'], 
#              num_inputs = layer_dict['Neuron_arch']['Inputs_number'],
#              IO_dict_list = [layer_dict['Neuron_arch']['IO']['shared_IO'], layer_dict['Neuron_arch']['IO']['unique_IO']]
#              )}
# ARCHITECTURE arch OF  {MAC_name}  IS
# {MAC_signals}
# BEGIN
# {MAC_Tree_Txt_Gen_Barriers(ReceiveStr_list = [signal_receive_string_MAC, rst_receive_string_MAC, clk_receive_string_MAC])}
# END ARCHITECTURE;
# '''
# )
#     return vhd_txt

In [ ]:
print(MAC_Tree_Barriers_TxtGen(MAC_name='MAC',
                               name_with_parameters=True,
                               rst_space=3*4,
                               clk_space=4*4,
                               layer_dict=layer_dict_hidden))

tentativa dic 1
LIBRARY ieee;
USE ieee.std_logic_1164.ALL;
USE ieee.numeric_std.ALL;
USE ieee.math_real.ALL;
ENTITY  MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0 IS
  PORT (
    clk, rst: IN STD_LOGIC;
    bias: IN signed(7 DOWNTO 0);
    x1, x2, x3: IN signed(7 DOWNTO 0);
    w1, w2, w3: IN signed(7 DOWNTO 0);
    ----------------------------------------------
    y: OUT signed(7 DOWNTO 0)
    );
end ENTITY;
ARCHITECTURE arch OF  MAC_comb_barriers_3n_8bit_unsigned_mul0_v0_add0_v0  IS
------------- SINAIS -------------

        SIGNAL sbias : signed(7 DOWNTO 0);
	SIGNAL sum_all : signed( 15 DOWNTO 0);
	SIGNAL sx1, sx2, sx3: signed(7 DOWNTO 0); 
	SIGNAL sw1, sw2, sw3: signed(7 DOWNTO 0); 
	SIGNAL soma_1, soma_2, soma_3: signed(15 DOWNTO 0); 
	SIGNAL mult_1, mult_2, mult_3: signed(15 DOWNTO 0); 
    

BEGIN

  sx1 <= x1;
  sx2 <= x2;
  sx3 <= x3;
  sw1 <= w1;
  sw2 <= w2;
  sw3 <= w3;
  sbias <= bias;
	------------------
	PROCESS (rst, clk) 
	BEGIN
		IF (rst = '1') THEN 
          